# Cell 0: Notebook Header & Documentation
# Description: Provides context and instructions for this notebook.

## Notebook Title: Ablation Study - Biological Analysis of Dynamic Regions

### Purpose and Context

*   **Goal:** To perform functional enrichment analysis on the **dynamically identified regions** for *each* simulation run in the Ablation Study (`ablation_01` through `ablation_07`). This goes beyond analyzing only the static Red Region from the baseline run and assesses the biological relevance of the sustained dynamic activity patterns generated by different ruleset variations.
*   **Contribution:** Loads the lists of dynamic region node IDs (as calculated and saved by `ablation_09`). Maps these IDs to gene symbols. Performs GO Biological Process enrichment analysis for the dynamic region of each run. Compiles and presents the enrichment findings in a comparative table.
*   **Inputs:**
    *   Requires the baseline configuration file (`baseline_config.json`) saved by `ablation_00`.
    *   Requires the text files containing the dynamic region node IDs for each run (`dynamic_region_nodes_run_label.txt`) generated by `ablation_09`.
*   **Outputs:**
    *   A dedicated analysis output folder (`biological_analysis_results/Dynamic_Biological_Analysis`).
    *   Saved full enrichment results CSVs for each run's dynamic region (e.g., `dynamic_region_run_label_enrichment_results.csv`).
    *   A summary CSV file containing the comparative enrichment metrics table across all runs.
    *   A markdown summary of the biological analysis findings for dynamic regions.

### How to Run

*   **Prerequisites:** Ensure `ablation_00_Setup_and_Definitions.ipynb` and ALL simulation notebooks (`ablation_01` through `ablation_07`) have been run successfully. Ensure `ablation_09_Dynamic_Analysis_Across_Runs.ipynb` has been run successfully to calculate and save the dynamic region node lists. Ensure the Canonical Helper Functions for biological analysis are defined in Cell 2 of THIS notebook.
*   **Configuration:** No user edits are required; Cell 1 loads the baseline config and defines specific analysis parameters. Cell 3 defines the mapping from analysis labels to run folders (matching `ablation_09`).
*   **Execution:** Run all cells sequentially from top to bottom (Cell 0 through Cell 8).
*   **Expected Runtime:** High, depends on the number of runs, size of dynamic regions, and the responsiveness of the STRING API and Enrichr server. This step can take significant time due to multiple API calls and enrichment analyses.

### Expected Results & Analysis (within this notebook)

*   This notebook loads the dynamic region node lists for each of the 7 simulation runs.
*   It maps the node IDs to gene symbols and performs GO BP enrichment analysis for each run's dynamic region gene list.
*   It compiles a table summarizing the enrichment results (e.g., number of significant terms, top term, its p-value) for the dynamic region of each run.
*   A final markdown summary discusses the biological relevance of the dynamic patterns generated by different ruleset variants based on these enrichment results. This table will be used for the final paper draft synthesis in `ablation_11`.

In [1]:
# Cell 1: Load Configuration and Define Analysis Parameters
# Description: Loads the baseline configuration to get directory paths and
#              defines the parameters for the consistent dynamic region analysis.
#              MODIFIED: Ensures BASE_EXPERIMENT_NAME, TARGET_NODE_ID, TARGET_NODE_NAME,
#              API/Enrichment params, etc. are set as globals.
#              MODIFIED: Standardized output directory global name to OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS.
#              FIXED: Corrected the check for OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS's existence.
#              FIXED: Defined INPUT_DIR_DYNAMIC_REGIONS before attempting to use it in the config dict.
#              FIXED: Ensure all necessary API/Enrichment parameters are extracted from baseline_config
#                     and set as globals before being used in the analysis_config_dict.
#              FIXED: Ensure BASE_EXPERIMENT_NAME is set as a global in the outer scope of Cell 1.
#              FIXED: Ensure SPECIES_ID and other API/Enrichment parameters are extracted and set as globals.
#              FIXED: Ensure OUTPUT_DIR_DYNAMIC_ANALYSIS is set as a global variable.

import os
import json
import time
import traceback
import warnings
import numpy as np # Needed for np.nan checks later

print(f"\n--- Cell 1: Load Configuration and Define Analysis Parameters ({time.strftime('%Y-%m-%d %H:%M:%S')}) ---")

# --- Load Baseline Configuration ---
config_load_error = False
baseline_config = {}
setup_output_dir_load = os.path.join("simulation_results", "Ablation_Setup_Files")

try:
    config_path_load = os.path.join(setup_output_dir_load, "baseline_config.json")
    if not os.path.exists(config_path_load): raise FileNotFoundError(f"Baseline config file not found: {config_path_load}. Run ablation_00.")
    with open(config_path_load, 'r') as f: baseline_config = json.load(f)
    print(f"  ✅ Loaded baseline configuration from: {config_path_load}")

    # Extract needed base parameters and set as GLOBALS
    OUTPUT_DIR_SIMULATIONS = baseline_config.get('OUTPUT_DIR', "simulation_results") # Base dir for sim outputs
    ANALYSIS_DIR_BASE = baseline_config.get('ANALYSIS_DIR', "biological_analysis_results") # Base dir for analysis outputs

    # --- Extract BASE_EXPERIMENT_NAME and set as GLOBAL ---
    # Set BASE_EXPERIMENT_NAME as a global directly in the outer scope of Cell 1
    BASE_EXPERIMENT_NAME = baseline_config.get('EXPERIMENT_NAME', 'string_ca_subgraph_AIFM1_CORRECTED')
    globals()['BASE_EXPERIMENT_NAME'] = BASE_EXPERIMENT_NAME
    print(f"  Base Experiment Name loaded and set globally: {BASE_EXPERIMENT_NAME}")
    # --- END MODIFIED ---


    # --- Extract other relevant globals from baseline config ---
    MASTER_SEED = baseline_config.get('MASTER_SEED', 42) # Keep seed for consistency if needed
    TARGET_NODE_ID = baseline_config.get('TARGET_NODE_ID') # Target ID for baselines
    TARGET_NODE_NAME = baseline_config.get('TARGET_NODE_NAME', 'TargetProtein') # Target Name

    # Set these as globals for helper functions
    globals()['MASTER_SEED'] = MASTER_SEED
    globals()['TARGET_NODE_ID'] = TARGET_NODE_ID
    globals()['TARGET_NODE_NAME'] = TARGET_NODE_NAME
    globals()['OUTPUT_DIR_SIMULATIONS'] = OUTPUT_DIR_SIMULATIONS
    globals()['ANALYSIS_DIR_BASE'] = ANALYSIS_DIR_BASE
    # --- END Extract and Set GLOBALS ---

    # --- MODIFIED: Extract and set all necessary API/Enrichment parameters as GLOBALS ---
    # These are needed by the biological analysis helpers loaded later
    SPECIES_ID = baseline_config.get('SPECIES_ID', "9606")
    STRING_API_URL = baseline_config.get('STRING_API_URL', "https://string-db.org/api")
    STRING_MAPPING_ENDPOINT = baseline_config.get('STRING_MAPPING_ENDPOINT', "/tsv/get_string_ids")
    BATCH_SIZE = baseline_config.get('BATCH_SIZE', 100)
    MAX_RETRIES = baseline_config.get('MAX_RETRIES', 5)
    RETRY_DELAY = baseline_config.get('RETRY_DELAY', 7)
    GO_ENRICHMENT_LIBRARY = baseline_config.get('GO_ENRICHMENT_LIBRARY', 'GO_Biological_Process_2023')
    TOP_N_ENRICHMENT_TERMS = baseline_config.get('TOP_N_ENRICHMENT_TERMS', 15)

    globals()['SPECIES_ID'] = SPECIES_ID
    globals()['STRING_API_URL'] = STRING_API_URL
    globals()['STRING_MAPPING_ENDPOINT'] = STRING_MAPPING_ENDPOINT
    globals()['BATCH_SIZE'] = BATCH_SIZE
    globals()['MAX_RETRIES'] = MAX_RETRIES
    globals()['RETRY_DELAY'] = RETRY_DELAY
    globals()['GO_ENRICHMENT_LIBRARY'] = GO_ENRICHMENT_LIBRARY
    globals()['TOP_N_ENRICHMENT_TERMS'] = TOP_N_ENRICHMENT_TERMS
    # --- END MODIFIED ---


except FileNotFoundError as e: print(f"❌ ERROR: {e}"); config_load_error = True
except Exception as e: print(f"❌ Error loading config data: {e}"); traceback.print_exc(limit=1); config_load_error = True


if not config_load_error:
    # --- Define Consistent Dynamic Region Analysis Parameters ---
    # Based on user's clarified definition (time-averaged absolute change over window)

    # 1. Window: Use the last 20% of steps
    DYNAMIC_WINDOW_FRACTION = 0.20
    # 2. Metric: Time-averaged absolute state CHANGE over the window (Act/Inh only)
    DYNAMIC_METRIC_NAME = "Time-Avg Abs Change (|Act_t+1-Act_t|, |Inh_t+1-Inh_t|)" # Descriptive name
    DYNAMIC_METRIC_KEY = 'time_avg_abs_change' # Internal key
    # 3. Threshold: Above the 80th-percentile of this metric across all nodes in the final step
    DYNAMIC_THRESHOLD_TYPE = 'percentile'
    DYNAMIC_THRESHOLD_VALUE = 80 # 80th percentile

    # --- Set these parameters as GLOBALS ---
    globals()['DYNAMIC_WINDOW_FRACTION'] = DYNAMIC_WINDOW_FRACTION
    globals()['DYNAMIC_METRIC_NAME'] = DYNAMIC_METRIC_NAME
    globals()['DYNAMIC_METRIC_KEY'] = DYNAMIC_METRIC_KEY
    globals()['DYNAMIC_THRESHOLD_TYPE'] = DYNAMIC_THRESHOLD_TYPE
    globals()['DYNAMIC_THRESHOLD_VALUE'] = DYNAMIC_THRESHOLD_VALUE
    # --- END Set GLOBALS ---

    print("\n--- Consistent Dynamic Region Analysis Parameters ---")
    print(f"  Window: Last {DYNAMIC_WINDOW_FRACTION*100:.0f}% of steps")
    print(f"  Metric: {DYNAMIC_METRIC_NAME}")
    print(f"  Threshold: Above {DYNAMIC_THRESHOLD_VALUE}th percentile of metric values across nodes")


    # --- Define Output Directory for this Analysis Notebook ---
    # Standardized global name to OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS
    OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS = os.path.join(ANALYSIS_DIR_BASE, "Dynamic_Biological_Analysis")
    os.makedirs(ANALYSIS_DIR_BASE, exist_ok=True) # Ensure base analysis dir exists
    os.makedirs(OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS, exist_ok=True) # Ensure specific analysis dir exists
    globals()['OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS'] = OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS # Set as global
    print(f"\nAnalysis outputs will be saved in: {os.path.join(os.getcwd(), OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS)}") # Show full path

    # --- Define INPUT_DIR_DYNAMIC_REGIONS ---
    # This is the output directory of ablation_09
    INPUT_DIR_DYNAMIC_REGIONS = os.path.join(ANALYSIS_DIR_BASE, "Dynamic_Analysis_Across_Runs")
    # --- FIXED: Set INPUT_DIR_DYNAMIC_REGIONS as a global ---
    globals()['INPUT_DIR_DYNAMIC_REGIONS'] = INPUT_DIR_DYNAMIC_REGIONS
    # --- END FIXED ---

    # --- ADDED: Define OUTPUT_DIR_DYNAMIC_ANALYSIS as a global ---
    # In ablation_09, the OUTPUT_DIR_DYNAMIC_ANALYSIS variable points to the directory where ablation_09 saves its own outputs.
    # This is the same directory as INPUT_DIR_DYNAMIC_REGIONS for THIS notebook (ablation_10).
    # So, set this global to the same path as INPUT_DIR_DYNAMIC_REGIONS.
    globals()['OUTPUT_DIR_DYNAMIC_ANALYSIS'] = INPUT_DIR_DYNAMIC_REGIONS
    # --- END ADDED ---


    # --- Save Analysis Config ---
    analysis_config_save_path = os.path.join(OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS, "dynamic_biological_analysis_config.json")
    try:
        analysis_config_dict = {
             'INPUT_DIR_DYNAMIC_REGIONS': INPUT_DIR_DYNAMIC_REGIONS, # This path is used by Cell 4
             'SPECIES_ID': SPECIES_ID, # Use the variable name (now global)
             'STRING_API_URL': STRING_API_URL, # Use the variable name (now global)
             'STRING_MAPPING_ENDPOINT': STRING_MAPPING_ENDPOINT, # Use the variable name (now global)
             'BATCH_SIZE': BATCH_SIZE, # Use the variable name (now global)
             'MAX_RETRIES': MAX_RETRIES, # Use the variable name (now global)
             'RETRY_DELAY': RETRY_DELAY, # Use the variable name (now global)
             'GO_ENRICHMENT_LIBRARY': GO_ENRICHMENT_LIBRARY, # Use the variable name (now global)
             'TOP_N_ENRICHMENT_TERMS': TOP_N_ENRICHMENT_TERMS, # Use the variable name (now global)
             'ANALYSIS_DIR_BASE': ANALYSIS_DIR_BASE,
             'OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS': OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS,
             'TARGET_NODE_ID': TARGET_NODE_ID, # Use the variable name
             'TARGET_NODE_NAME': TARGET_NODE_NAME, # Use the variable name
             'MASTER_SEED': MASTER_SEED # Include seed for record-keeping
        }
        with open(analysis_config_save_path, 'w') as f:
            json.dump(analysis_config_dict, f, indent=4, default=str) # Use default=str for numpy types
        print(f"   ✅ Saved analysis configuration to {analysis_config_save_path}")
    except Exception as e: print(f"   ⚠️ Warning: Could not save analysis configuration: {e}")

else:
     print("\n❌ Configuration loading failed. Analysis cannot proceed.")


print("\nCell 1: Configuration loaded and analysis parameters defined.")


--- Cell 1: Load Configuration and Define Analysis Parameters (2025-04-28 21:18:25) ---
  ✅ Loaded baseline configuration from: simulation_results/Ablation_Setup_Files/baseline_config.json
  Base Experiment Name loaded and set globally: string_ca_subgraph_AIFM1_CORRECTED

--- Consistent Dynamic Region Analysis Parameters ---
  Window: Last 20% of steps
  Metric: Time-Avg Abs Change (|Act_t+1-Act_t|, |Inh_t+1-Inh_t|)
  Threshold: Above 80th percentile of metric values across nodes

Analysis outputs will be saved in: /home/irbsurfer/Projects/Novyte/Emergenics/production/emergenics/1_NetworkIStheComputation/ablation_study/biological_analysis_results/Dynamic_Biological_Analysis
   ✅ Saved analysis configuration to biological_analysis_results/Dynamic_Biological_Analysis/dynamic_biological_analysis_config.json

Cell 1: Configuration loaded and analysis parameters defined.


In [2]:
# Cell 1.1: Canonical Helper Functions for Dynamic Biological Analysis
# Description: Defines ALL helper functions required by subsequent cells in THIS notebook
#              (ablation_10). Includes functions for loading histories, calculating
#              dynamic region metrics, loading static baselines, mapping IDs, running
#              enrichment, and plotting. These functions are self-contained within this notebook.
#              MODIFIED: Includes function for calculating time-averaged absolute CHANGE.
#              MODIFIED: Added warning suppression in load_history_dfs for non-numeric checks.
#              MODIFIED: Corrected logic in load_static_baseline_nodes to handle file not found more explicitly
#                        and avoid printing the "Loaded X nodes for baseline check" message if the file isn't used.

import pandas as pd
import numpy as np
import os
import warnings
import pickle # Needed for loading pkl files
import traceback
import time # Needed for time.strftime if functions use it
import requests # Needed for API calls in map_string_ids_to_gene_symbols_api
import io # Needed for io.StringIO in API mapping
from tqdm.notebook import tqdm # Use notebook version for progress bars

# Ensure gseapy is available (checked in Cell 0 or earlier)
try:
    import gseapy as gp # Rename to gp for consistency
except ImportError:
    gp = None # Set to None to allow checks later


print(f"\n--- Cell 1.1: Defining Canonical Helper Functions for Dynamic Biological Analysis ({time.strftime('%Y-%m-%d %H:%M:%S')}) ---")


# === HELPER FUNCTIONS COPIED FROM ABLATION_09 CELL 1.1 (DYNAMIC ANALYSIS) ===

# --- Helper Function: Load History DataFrames from Run Folder ---
# COPIED from ablation_09 Cell 1.1
def load_history_dfs(run_folder_name, base_sim_output_dir):
    """
    Loads activation and inhibition history DataFrames from a simulation run folder.
    Returns (act_df, inh_df) or (None, None) on failure.
    Looks for filenames like 'activation_history_analysis.csv' or 'activation_history.csv'.
    Suppresses UserWarnings during non-numeric dtype checks.
    """
    act_df = None; inh_df = None
    run_output_dir = os.path.join(base_sim_output_dir, run_folder_name)
    # Prioritize 'analysis' suffixed files, fall back to standard names
    act_paths = [os.path.join(run_output_dir, "activation_history_analysis.csv"), os.path.join(run_output_dir, "activation_history.csv")]
    inh_paths = [os.path.join(run_output_dir, "inhibition_history_analysis.csv"), os.path.join(run_output_dir, "inhibition_history.csv")]

    try:
        found_act_path = next((p for p in act_paths if os.path.exists(p)), None)
        if found_act_path:
            # Use float dtype hint during read for performance/robustness if possible
            act_df = pd.read_csv(found_act_path, index_col=0, dtype=float, low_memory=False)
            # print(f"    Loaded Act history from: {os.path.basename(found_act_path)}") # Too verbose
        else: warnings.warn(f"    Act history not found for {run_folder_name}.")

        found_inh_path = next((p for p in inh_paths if os.path.exists(p)), None)
        if found_inh_path:
             inh_df = pd.read_csv(found_inh_path, index_col=0, dtype=float, low_memory=False)
             # print(f"    Loaded Inh history from: {os.path.basename(found_inh_path)}") # Too verbose
        else: warnings.warn(f"    Inh history not found for {run_folder_name}.")

        if act_df is None or inh_df is None: return None, None # Return None if either failed to load
        if act_df.empty or inh_df.empty: warnings.warn(f"    Loaded history DF(s) empty for {run_folder_name}."); return None, None
        if act_df.shape != inh_df.shape: warnings.warn(f"    History DF shape mismatch for {run_folder_name}: Act={act_df.shape}, Inh={inh_df.shape}. Using common steps/nodes.");
        # Harmonize dataframes based on common indices and columns
        common_indices = act_df.index.intersection(inh_df.index)
        common_columns = act_df.columns.intersection(inh_df.columns)
        act_df = act_df.loc[common_indices, common_columns]
        inh_df = inh_df.loc[common_indices, common_columns]
        if act_df.empty or inh_df.empty: warnings.warn(f"    DFs became empty after harmonization for {run_folder_name}."); return None, None

        # --- MODIFIED: Add warning suppression around the numeric dtype check ---
        with warnings.catch_warnings():
            warnings.simplefilter("ignore") # Suppress UserWarnings (like 'Non-numeric data detected')
            # Check for potential non-numeric data after loading - this check can raise UserWarnings
            # if there are mixed types or pandas isn't sure. The dtype=float hint helps.
            if not pd.api.types.is_numeric_dtype(act_df.values) or not pd.api.types.is_numeric_dtype(inh_df.values):
                # If despite dtype=float, it's still not purely numeric, try coercion.
                 # The original warning message will be suppressed by the catch_warnings block.
                 print(f"    Attempting coercion to numeric for history DFs for {run_folder_name}.")
                 act_df = act_df.apply(pd.to_numeric, errors='coerce')
                 inh_df = inh_df.apply(pd.to_numeric, errors='coerce')
                 if act_df.isnull().all().all() or inh_df.isnull().all().all():
                      warnings.warn(f"    Coercion failed, DFs are all NaN for {run_folder_name}. Skipping.")
                      return None, None # Return None if coercion fails completely
        # --- END MODIFIED ---


        return act_df, inh_df # Return successfully loaded/harmonized DFs

    except FileNotFoundError:
         # This case is covered by the checks inside the try block
         return None, None
    except Exception as e:
        print(f"   ❌ Error loading history DataFrames for {run_folder_name}: {e}")
        traceback.print_exc(limit=1)
        return None, None # Return None on any loading/processing error

# --- Helper Function: Calculate Time-Averaged Absolute State Change Metric ---
# COPIED from ablation_09 Cell 1.1
# MODIFIED: Implements the user's specified metric calculation.
# MODIFIED: Accesses parameters from GLOBALS as expected.
def calculate_time_avg_abs_change_metric(act_history_df, inh_history_df):
    """
    Calculates the time-averaged absolute change in Act/Inh for each node
    over the final window, based on the user's defined metric.
    Returns a pandas Series {node_id: metric_value} or None.
    """
    # Access parameters from GLOBALS
    window_fraction = globals().get('DYNAMIC_WINDOW_FRACTION', 0.20)
    metric_key = globals().get('DYNAMIC_METRIC_KEY', 'time_avg_abs_change') # For internal verification

    if act_history_df is None or inh_history_df is None or act_history_df.empty or inh_history_df.empty:
        warnings.warn(f"    Cannot calculate '{metric_key}': Input DataFrames are missing or empty.")
        return None

    if not act_history_df.index.equals(inh_history_df.index) or not act_history_df.columns.equals(inh_history_df.columns):
        warnings.warn(f"    Act/Inh DataFrames indices/columns do not match, using intersection.")
        common_indices = act_history_df.index.intersection(inh_history_df.index)
        common_columns = act_history_df.columns.intersection(inh_history_df.columns)
        act_history_df = act_history_df.loc[common_indices, common_columns]
        inh_history_df = inh_history_df.loc[common_indices, common_columns]
        if act_history_df.empty:
             warnings.warn(f"    DFs became empty after harmonization, cannot calculate '{metric_key}'.")
             return pd.Series(np.nan, index=[]) # Return empty series
    # Else DFs should be aligned


    num_steps_hist = len(act_history_df)
    if num_steps_hist < 2:
        warnings.warn(f"    History has < 2 steps ({num_steps_hist}), cannot calculate state change.")
        return pd.Series(np.nan, index=act_history_df.columns) # Return Series of NaN keyed by node ID


    # Calculate step-wise absolute change for Act and Inh
    # Use diff() which calculates change between adjacent steps
    # The first row of diff() will be NaN
    abs_change_act = act_history_df.diff().abs()
    abs_change_inh = inh_history_df.diff().abs()

    # Combine absolute changes (element-wise maximum)
    # Handle potential dimension mismatches if harmonization failed strangely
    if not abs_change_act.index.equals(abs_change_inh.index) or not abs_change_act.columns.equals(abs_change_inh.columns):
         warnings.warn("    Absolute change DFs indices/columns mismatch after diff().")
         # Attempt re-harmonization based on common indices/columns again
         common_indices = abs_change_act.index.intersection(abs_change_inh.index)
         common_columns = abs_change_act.columns.intersection(abs_change_inh.columns)
         abs_change_act = abs_change_act.loc[common_indices, common_columns]
         abs_change_inh = abs_change_inh.loc[common_indices, common_columns]
         if abs_change_act.empty:
             warnings.warn("    Change DFs became empty after harmonization.")
             return pd.Series(np.nan, index=[])


    abs_change_max = pd.DataFrame(np.maximum(abs_change_act.values, abs_change_inh.values), index=abs_change_act.index, columns=abs_change_act.columns)


    # Determine the window start index
    # Start window from the beginning of the history if it's shorter than window size
    window_length = max(1, int(window_fraction * num_steps_hist)) # Ensure window is at least 1 step
    # Window should be applied to the *changes*, which start from step 1 (index 1)
    # So the number of change steps is num_steps_hist - 1
    num_change_steps = num_steps_hist - 1
    window_start_change_index = max(1, num_change_steps - window_length + 1) # Index in abs_change_max DataFrame

    # Select the window from the changes DataFrame (iloc uses integer index)
    window_changes = abs_change_max.iloc[window_start_change_index :]

    if window_changes.empty:
         warnings.warn(f"    Calculated window is empty ({window_length} steps from {num_steps_hist} total). Cannot calculate time-averaged change.")
         # Return Series of NaN keyed by node ID, matching original DataFrame columns
         return pd.Series(np.nan, index=act_history_df.columns)


    # Calculate the time-averaged change over the window for each node (mean across time steps)
    # Use .mean(axis=0) on the DataFrame for mean across rows (time) for each column (node)
    # Handle potential NaNs within the window data if they exist
    time_averaged_metric_per_node = window_changes.mean(axis=0) # Mean across rows (time)

    # Return as a pandas Series indexed by node ID
    return time_averaged_metric_per_node


# --- Helper Function: Identify Dynamic Nodes based on Metric ---
# COPIED from ablation_09 Cell 1.1
# MODIFIED: Implements the user's specified thresholding logic.
# MODIFIED: Accesses parameters from GLOBALS as expected.
def identify_dynamic_nodes(node_metric_series):
    """
    Identifies nodes whose metric value is above the percentile threshold.
    Returns a list of node IDs.
    """
    # Access parameters from GLOBALS
    threshold_type = globals().get('DYNAMIC_THRESHOLD_TYPE', 'percentile')
    threshold_value = globals().get('DYNAMIC_THRESHOLD_VALUE', 80)
    metric_key = globals().get('DYNAMIC_METRIC_KEY', 'metric_value') # For warning messages

    if node_metric_series is None or node_metric_series.empty:
        warnings.warn(f"    Cannot identify dynamic nodes: Input metric series is missing or empty.")
        return []

    # Exclude NaN values from the metric distribution for threshold calculation
    valid_metric_values = node_metric_series.dropna().values

    if len(valid_metric_values) == 0:
        warnings.warn(f"    All metric values are NaN for '{node_metric_series.name if hasattr(node_metric_series,'name') else 'Series'}'. Cannot determine threshold or dynamic nodes.")
        return []
    if len(valid_metric_values) < 2 and threshold_type == 'percentile':
         warnings.warn(f"    Fewer than 2 valid metric values ({len(valid_metric_values)}) for '{node_metric_series.name if hasattr(node_metric_series,'name') else 'Series'}', percentile threshold unreliable/impossible. Using median as threshold fallback.")
         # Fallback to median if percentile is requested but insufficient data
         threshold_type = 'absolute'
         threshold_value = np.median(valid_metric_values) if valid_metric_values.size > 0 else 0.0


    actual_threshold = np.nan # Initialize

    if threshold_type == 'percentile':
        percentile = threshold_value # Use the value directly (e.g., 80)
        if not (0 <= percentile <= 100):
             warnings.warn(f"    Percentile threshold value out of bounds (0-100): {percentile}. Using 80th percentile.")
             percentile = 80
        try:
            actual_threshold = np.percentile(valid_metric_values, percentile)
            # print(f"    Using {percentile}th percentile threshold: {actual_threshold:.6f}") # Print in calling cell
        except Exception as e:
             warnings.warn(f"    Error calculating percentile threshold: {e}. Cannot identify dynamic nodes.")
             return [] # Return empty list on error

    elif threshold_type == 'absolute':
        actual_threshold = threshold_value # Use the value directly
        # print(f"    Using absolute threshold: {actual_threshold:.6f}") # Print in calling cell
    else:
        warnings.warn(f"    Unknown threshold type: '{threshold_type}'. Cannot identify dynamic nodes.")
        return [] # Return empty list on unknown type

    if pd.isna(actual_threshold): # Should be set by now, but check for safety
         warnings.warn("    Actual threshold value is NaN. Cannot identify dynamic nodes.")
         return []

    # Identify nodes whose metric value is >= the calculated/determined threshold
    # Use .loc to apply threshold to the Series and get the index (node IDs)
    # Handle potential NaNs in the original series by filling temporarily for comparison >= threshold
    dynamic_nodes_series = node_metric_series[np.nan_to_num(node_metric_series, nan=-np.inf) >= actual_threshold]

    # Return the list of node IDs
    return dynamic_nodes_series.index.tolist()


# === HELPER FUNCTIONS COPIED FROM ABLATION_08 CELL 2 (BIOLOGICAL ANALYSIS) ===

# --- Helper Function: Load STRING IDs from File (Duplicate, exists above, keep only one) ---
# The definition is identical to the one above in this combined block. Keep only one.
# def load_string_ids_from_txt(filename):
#     """Loads STRING IDs directly from a text file (one ID per line)."""
#     # ... (identical code) ...
#     pass # Definition is above


# --- Helper Function: Map STRING IDs to Gene Symbols via API with Caching ---
# COPIED from ablation_08 Cell 2
# MODIFIED: Use GLOBALS for config parameters (SPECIES_ID, API_URL, etc.)
# MODIFIED: Fixed SyntaxError in map_string_ids_to_gene_symbols_api.
# MODIFIED: Fixed NameError in map_string_ids_to_gene_symbols_api by using the correct variable name.
# MODIFIED: Corrected output dir variable name to OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS.
def map_string_ids_to_gene_symbols_api(string_ids, description="IDs"):
    """Maps STRING IDs (e.g., 9606.ENSP...) to Gene Symbols using the STRING API with caching."""
    # Access globals for configuration
    species_id = globals().get('SPECIES_ID', '9606')
    api_url = globals().get('STRING_API_URL', 'https://string-db.org/api')
    mapping_endpoint = globals().get('STRING_MAPPING_ENDPOINT', '/tsv/get_string_ids')
    batch_size = globals().get('BATCH_SIZE', 100)
    max_retries = globals().get('MAX_RETRIES', 3)
    retry_delay = globals().get('RETRY_DELAY', 5)
    # MODIFIED: Use the correct global variable for this notebook's output directory
    output_dir_analysis_base = globals().get('OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS', '.')
    # END MODIFIED

    if not string_ids: print(f"⚠️ No input STRING IDs provided for {description}. Skipping mapping."); return []
    print(f"Mapping {len(string_ids)} {description} IDs...")

    # --- Caching Logic ---
    # Use a consistent cache file name within this notebook's output directory
    mapping_cache_file = os.path.join(output_dir_analysis_base, 'string_id_to_gene_symbol_map_cache.pkl')
    cached_map = {}
    if os.path.exists(mapping_cache_file):
        try:
             with open(mapping_cache_file, 'rb') as f_cache: cached_map = pickle.load(f_cache)
             if not isinstance(cached_map, dict): cached_map = {}
             else: print(f"  ℹ️ Loaded {len(cached_map)} mappings from cache: {mapping_cache_file}")
        except Exception as e_cache: print(f"  ⚠️ Warning: Could not load mapping cache: {e_cache}. Will rebuild.")

    # --- Identify IDs needing API call ---
    ids_needing_map = sorted(list(set(string_ids) - set(cached_map.keys())))
    mapped_symbols_dict = cached_map.copy() # Start with cache. THIS IS THE CORRECT VARIABLE NAME.

    if identifiers_to_map:
        print(f"  🌐 Mapping {len(identifiers_to_map)} IDs via STRING API...")
        identifiers_to_map_api = [sid for sid in ids_needing_map if isinstance(sid, str) and sid.startswith(str(species_id) + '.')]
        if not identifiers_to_map_api: print("    No valid STRING IDs (starting with species ID) found for mapping via API.");
        else:
            api_map_url = f"{api_url}{mapping_endpoint}"; num_batches = (len(identifiers_to_map_api) + batch_size - 1)//batch_size;
            use_tqdm = num_batches > 1;
            batch_iterator = tqdm(range(num_batches), desc=f"API Mapping ({description})", leave=False) if use_tqdm else range(num_batches);
            api_errors = 0;
            for i in batch_iterator:
                 batch = identifiers_to_map_api[i*batch_size:(i+1)*batch_size];
                 payload = {'identifiers': "\r".join(batch), 'species': species_id, 'limit': 1, 'echo_query': 1, 'caller_identity': 'NetworkAutomatonSim/1.0'};
                 retries = 0; success = False; response = None;
                 while retries < max_retries and not success:
                      try:
                           response = requests.post(api_map_url, data=payload, timeout=60);
                           response.raise_for_status();
                           content_type = response.headers.get('Content-Type','');
                           if 'text/tab-separated-values' in content_type:
                                text = response.text;
                                if text and text.strip():
                                     mapping_df = pd.read_csv(io.StringIO(text), sep='\t')
                                     if 'preferredName' in mapping_df.columns and 'queryItem' in mapping_df.columns:
                                          for qid, name in zip(mapping_df['queryItem'], mapping_df['preferredName']):
                                               if pd.notna(name) and name: mapped_symbols_dict[qid] = str(name)
                                     else: warnings.warn(f"API response missing expected columns ('preferredName', 'queryItem') in batch {i+1}."); api_errors += 1; break
                                success = True;
                           else: warnings.warn(f"Unexpected API format batch {i+1}: {content_type}. Retrying..."); retries += 1; time.sleep(retry_delay*(1+retries));
                      except requests.exceptions.Timeout:
                           retries += 1;
                           print(f"\n   Timeout on API call (Batch {i+1}, Retry {retries}/{max_retries}). Sleeping...")
                           time.sleep(retry_delay * (1 + retries));
                      except requests.exceptions.RequestException as e:
                           retries += 1;
                           print(f"   API Request Error (Batch {i+1}, Retry {retries}/{max_retries}): {e}. Sleeping...")
                           time.sleep(retry_delay * (1 + retries));
                      except pd.errors.EmptyDataError:
                           success = True;
                      except Exception as e:
                           retries += 1;
                           print(f"   Unexpected Error Processing API Response (Batch {i+1}, Retry {retries}/{max_retries}): {e}")
                           traceback.print_exc(limit=1)
                           time.sleep(retry_delay * (1 + retries));
                      if retries >= max_retries and not success:
                           print(f"   ❌ API call failed for Batch {i+1} after {retries} retries.")
                           api_errors += 1
            if api_errors > 0: warnings.warn(f"🚨 API mapping failed for {api_errors}/{num_batches} batches after retries.")
            try:
                with open(mapping_cache_file, 'wb') as f_cache: pickle.dump(mapped_symbols_dict, f_cache) # Use mapped_symbols_dict
                print(f"  ✅ Updated mapping cache saved ({len(mapped_symbols_dict)} total mappings).") # Use mapped_symbols_dict
            except Exception as e_save_cache: print(f"  ⚠️ Error saving updated mapping cache: {e_save_cache}")
    else: print("  All required IDs found in cache.");

    mapped_symbols_list = []
    for sid in string_ids:
        mapped_val = mapped_symbols_dict.get(sid, sid) # Use mapped_symbols_dict
        if isinstance(mapped_val, str) and not ('.' in mapped_val and mapped_val.split('.')[0] == str(species_id)):
             mapped_symbols_list.append(mapped_val)
    # print(f"\n✅ Mapping complete. Found {len(unique_gene_symbols)} unique Gene Symbols for {description}.") # Printed in calling loop
    return sorted(list(set(mapped_symbols_list))) # Return sorted unique list


# --- Helper Function: Run Enrichment Analysis ---
# COPIED from ablation_08 Cell 2
# MODIFIED: Use GLOBALS for gene_sets and top_n defaults.
def run_enrichment_simple(gene_list, description):
     """Runs gseapy enrichment analysis."""
     # Access globals for configuration
     gene_sets = globals().get('GO_ENRICHMENT_LIBRARY', 'GO_Biological_Process_2023')
     top_n = globals().get('TOP_N_ENRICHMENT_TERMS', 15)

     if gp is None: print("❌ Cannot run enrichment: gseapy not available."); return None
     if not isinstance(gene_list, list) or not all(isinstance(g, str) for g in gene_list):
         print(f"⚠️ Skip enrichment: Input gene_list is not a list of strings for '{description}'.")
         return None
     cleaned_gene_list = [g for g in gene_list if g]
     if not cleaned_gene_list or len(cleaned_gene_list) < 3:
         # print(f"⚠️ Skip enrichment '{description}': Needs >= 3 non-empty genes (found {len(cleaned_gene_list)})."); # Printed in calling loop
         return pd.DataFrame()

     print(f"--- 📊 Performing Enrichment: {description} ({len(cleaned_gene_list)} genes) using '{gene_sets}' ---")
     try:
          libs = gp.get_library_name(organism='Human')
          valid_libs = [gene_sets] if gene_sets in libs else [] # Only use the specified library if valid
          if not valid_libs:
              print(f"❌ Invalid or unavailable gene_set: '{gene_sets}'. Available examples: {libs[:5]}")
              return pd.DataFrame()

          enr = gp.enrichr(gene_list=cleaned_gene_list, gene_sets=valid_libs, organism='Human', outdir=None, cutoff=0.05) # Use gp alias

          if enr is None or not hasattr(enr, 'results') or enr.results.empty:
               # print(f"ℹ️ No significant terms found for '{description}' using library: '{gene_sets}'."); # Printed in calling loop
               return pd.DataFrame()

          results_df = enr.results.sort_values('Adjusted P-value').reset_index(drop=True)
          # print(f"🧬 Top {min(top_n, len(results_df))} Enriched Terms ('{gene_sets}'):") # Printed in calling loop
          # Format for display - only if needed, not for return value
          # display_df=results_df[['Term', 'Adjusted P-value', 'Overlap', 'Genes']].head(top_n).copy()
          # display_df['Adjusted P-value'] = display_df['Adjusted P-value'].map('{:.2e}'.format)
          # display_df['Genes'] = display_df['Genes'].fillna('').astype(str).str.split(';').str[:5].apply(lambda x: ';'.join(x)+('...' if len(x)==5 else ''))
          # print(display_df.to_string(index=False)); print("-" * 70);

          return results_df # Return the full DataFrame
     except requests.exceptions.RequestException as e:
          print(f"❌ GSEAPY Enrichr Connection Error for '{description}': {e}");
          return None # Indicate error
     except Exception as e:
          print(f"❌ Unexpected enrichment error for '{description}': {e}");
          traceback.print_exc(limit=2);
          return None # Indicate error


# --- Helper Function: Plot Enrichment Results ---
# COPIED from ablation_08 Cell 2
# MODIFIED: Use GLOBALS for top_n and output_dir defaults.
# MODIFIED: Removed automatic saving and displaying; calling cell will handle that.
def create_enrichment_plot_simple(results_df, title):
    """
    Generates a horizontal bar plot for enrichment results.
    Returns the matplotlib Figure object or None. Does NOT save or display automatically.
    """
    # Access globals for configuration
    top_n = globals().get('TOP_N_ENRICHMENT_TERMS', 15)

    if not isinstance(results_df, pd.DataFrame) or results_df.empty: print(f"⚠️ Plot function skipped for '{title}': No data."); return None
    pval_col = 'Adjusted P-value'; term_col = 'Term'
    if pval_col not in results_df.columns or term_col not in results_df.columns: print(f"❌ Plot Error for '{title}': Missing columns '{pval_col}' or '{term_col}'."); return None

    print(f"--- Generating Enrichment Plot: {title} ---")
    plot_data = results_df.dropna(subset=[pval_col]).sort_values(by=pval_col, ascending=True).head(top_n).copy();
    if plot_data.empty: print("ℹ️ No terms left to plot after filtering/sorting."); return None
    epsilon = np.finfo(float).tiny
    plot_data['-log10 Adj P'] = -np.log10(plot_data[pval_col].replace(0, epsilon) + epsilon)
    plot_data.sort_values('-log10 Adj P', ascending=True, inplace=True)

    num_terms_plotted = len(plot_data); fig_height = max(6, num_terms_plotted * 0.5)
    fig, ax = plt.subplots(figsize=(10, fig_height))
    sns.barplot(x='-log10 Adj P', y=term_col, data=plot_data, palette='viridis_r', ax=ax, orient='h')
    ax.set_title(title, fontsize=13); ax.set_xlabel(f'-log10 ({pval_col})', fontsize=11); ax.set_ylabel('');
    ax.tick_params(axis='y', labelsize=10); ax.tick_params(axis='x', labelsize=10)
    ax.xaxis.grid(True, linestyle='--', alpha=0.6); ax.yaxis.grid(False)
    xlim_max = ax.get_xlim()[1]; text_offset = xlim_max * 0.01;
    for i, bar_val in enumerate(plot_data['-log10 Adj P']): ax.text(bar_val + text_offset , i, f'{bar_val:.2f}', va='center', ha='left', fontsize=9)
    plt.tight_layout()

    # Return the figure handle
    return fig


print("\n ✅ Cell 1.1: Canonical biological analysis helper functions defined (Using GLOBALS for config).")


--- Cell 1.1: Defining Canonical Helper Functions for Dynamic Biological Analysis (2025-04-28 21:18:27) ---

 ✅ Cell 1.1: Canonical biological analysis helper functions defined (Using GLOBALS for config).


In [3]:
# Cell 2: Canonical Helper Functions for Biological Analysis
# Description: Defines ALL necessary helper functions for loading IDs, mapping to
#              gene symbols via STRING API, running GSEAPY enrichment, and plotting
#              enrichment results. These functions are self-contained within this notebook.
#              MODIFIED: Ensure functions use GLOBALS for config parameters.
#              COPIED from ablation_08 Cell 2.

import pandas as pd
import requests
import time
import io
from tqdm.notebook import tqdm # Use notebook version for progress bars
import os
import warnings
import pickle # Needed for loading pkl files
import traceback
import numpy as np # For np.nan

# Ensure gseapy is available (checked in Cell 0 or earlier)
try:
    import gseapy as gp # Rename to gp for consistency
except ImportError:
    gp = None # Set to None to allow checks later


print(f"\n--- Cell 2: Defining Canonical Helper Functions for Biological Analysis ({time.strftime('%Y-%m-%d %H:%M:%S')}) ---")


# --- Helper Function: Load STRING IDs from File ---
# COPIED from ablation_08 Cell 2
def load_string_ids_from_txt(filename):
    """Loads STRING IDs directly from a text file (one ID per line)."""
    string_ids = []
    try:
        with open(filename, 'r') as f:
            string_ids = [line.strip() for line in f if line.strip()]
        print(f"  📄 Loaded {len(string_ids)} STRING IDs from '{os.path.basename(filename)}'")
        if not string_ids: warnings.warn(f"Loaded list from '{filename}' is empty.")
        return string_ids
    except FileNotFoundError: print(f"  ❌ Error: File not found at {filename}"); return []
    except Exception as e: print(f"  ❌ Error loading data from {filename}: {e}"); return []


# --- Helper Function: Map STRING IDs to Gene Symbols via API with Caching ---
# COPIED from ablation_08 Cell 2
def map_string_ids_to_gene_symbols_api(string_ids, description="IDs"):
    """Maps STRING IDs (e.g., 9606.ENSP...) to Gene Symbols using the STRING API with caching."""
    # Access globals for configuration
    species_id = globals().get('SPECIES_ID', '9606')
    api_url = globals().get('STRING_API_URL', 'https://string-db.org/api')
    mapping_endpoint = globals().get('STRING_MAPPING_ENDPOINT', '/tsv/get_string_ids')
    batch_size = globals().get('BATCH_SIZE', 100)
    max_retries = globals().get('MAX_RETRIES', 3)
    retry_delay = globals().get('RETRY_DELAY', 5)
    output_dir_analysis_base = globals().get('OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS', '.') # Use this notebook's output dir

    if not string_ids: print(f"⚠️ No input STRING IDs provided for {description}. Skipping mapping."); return []
    print(f"Mapping {len(string_ids)} {description} IDs...")

    # --- Caching Logic ---
    # Use a consistent cache file name within this notebook's output directory
    mapping_cache_file = os.path.join(output_dir_analysis_base, 'string_id_to_gene_symbol_map_cache.pkl')
    cached_map = {}
    if os.path.exists(mapping_cache_file):
        try:
             with open(mapping_cache_file, 'rb') as f_cache: cached_map = pickle.load(f_cache)
             if not isinstance(cached_map, dict): cached_map = {}
             else: print(f"  ℹ️ Loaded {len(cached_map)} mappings from cache: {mapping_cache_file}")
        except Exception as e_cache: print(f"  ⚠️ Warning: Could not load mapping cache: {e_cache}. Will rebuild.")

    # --- Identify IDs needing API call ---
    ids_needing_map = sorted(list(set(string_ids) - set(cached_map.keys())))
    mapped_symbols_dict = cached_map.copy()

    if ids_needing_map:
        print(f"  🌐 Mapping {len(ids_needing_map)} IDs via STRING API...")
        identifiers_to_map = [sid for sid in ids_needing_map if isinstance(sid, str) and sid.startswith(str(species_id) + '.')]
        if not identifiers_to_map: print("    No valid STRING IDs (starting with species ID) found for mapping via API.");
        else:
            api_map_url = f"{api_url}{mapping_endpoint}"; num_batches = (len(identifiers_to_map) + batch_size - 1)//batch_size;
            use_tqdm = num_batches > 1;
            batch_iterator = tqdm(range(num_batches), desc=f"API Mapping ({description})", leave=False) if use_tqdm else range(num_batches);
            api_errors = 0;
            for i in batch_iterator:
                 batch = identifiers_to_map[i*batch_size:(i+1)*batch_size];
                 payload = {'identifiers': "\r".join(batch), 'species': species_id, 'limit': 1, 'echo_query': 1, 'caller_identity': 'NetworkAutomatonSim/1.0'};
                 retries = 0; success = False; response = None;
                 while retries < max_retries and not success:
                      try:
                           response = requests.post(api_map_url, data=payload, timeout=60);
                           response.raise_for_status();
                           content_type = response.headers.get('Content-Type','');
                           if 'text/tab-separated-values' in content_type:
                                text = response.text;
                                if text and text.strip():
                                     mapping_df = pd.read_csv(io.StringIO(text), sep='\t')
                                     if 'preferredName' in mapping_df.columns and 'queryItem' in mapping_df.columns:
                                          for qid, name in zip(mapping_df['queryItem'], mapping_df['preferredName']):
                                               if pd.notna(name) and name: mapped_symbols_dict[qid] = str(name)
                                     else: warnings.warn(f"API response missing expected columns ('preferredName', 'queryItem') in batch {i+1}."); api_errors += 1; break
                                success = True;
                           else: warnings.warn(f"Unexpected API format batch {i+1}: {content_type}. Retrying..."); retries += 1; time.sleep(retry_delay*(1+retries));
                      except requests.exceptions.Timeout:
                           retries += 1;
                           print(f"\n   Timeout on API call (Batch {i+1}, Retry {retries}/{max_retries}). Sleeping...")
                           time.sleep(retry_delay * (1 + retries));
                      except requests.exceptions.RequestException as e:
                           retries += 1;
                           print(f"   API Request Error (Batch {i+1}, Retry {retries}/{max_retries}): {e}. Sleeping...")
                           time.sleep(retry_delay * (1 + retries));
                      except pd.errors.EmptyDataError:
                           success = True;
                      except Exception as e:
                           retries += 1;
                           print(f"   Unexpected Error Processing API Response (Batch {i+1}, Retry {retries}/{max_retries}): {e}")
                           traceback.print_exc(limit=1)
                           time.sleep(retry_delay * (1 + retries));
                      if retries >= max_retries and not success:
                           print(f"   ❌ API call failed for Batch {i+1} after {retries} retries.")
                           api_errors += 1
            if api_errors > 0: warnings.warn(f"🚨 API mapping failed for {api_errors}/{num_batches} batches after retries.")
            try:
                with open(mapping_cache_file, 'wb') as f_cache: pickle.dump(mapped_symbols_dict, f_cache)
                print(f"  ✅ Updated mapping cache saved ({len(mapped_symbols_dict)} total mappings).")
            except Exception as e_save_cache: print(f"  ⚠️ Error saving updated mapping cache: {e_save_cache}")
    else: print("  All required IDs found in cache.");

    mapped_symbols_list = []
    for sid in string_ids:
        mapped_val = mapped_symbols_dict.get(sid, sid)
        if isinstance(mapped_val, str) and not ('.' in mapped_val and mapped_val.split('.')[0] == str(species_id)):
             mapped_symbols_list.append(mapped_val)
    unique_gene_symbols = sorted(list(set(mapped_symbols_list)))
    # print(f"\n✅ Mapping complete. Found {len(unique_gene_symbols)} unique Gene Symbols for {description}.") # Printed in calling loop
    return unique_gene_symbols

# --- Helper Function: Run Enrichment Analysis ---
# COPIED from ablation_08 Cell 2
# MODIFIED: Ensure functions use GLOBALS for gene_sets and top_n defaults.
def run_enrichment_simple(gene_list, description):
     """Runs gseapy enrichment analysis."""
     # Access globals for configuration
     gene_sets = globals().get('GO_ENRICHMENT_LIBRARY', 'GO_Biological_Process_2023')
     top_n = globals().get('TOP_N_ENRICHMENT_TERMS', 15)

     if gp is None: print("❌ Cannot run enrichment: gseapy not available."); return None
     if not isinstance(gene_list, list) or not all(isinstance(g, str) for g in gene_list):
         print(f"⚠️ Skip enrichment: Input gene_list is not a list of strings for '{description}'.")
         return None
     cleaned_gene_list = [g for g in gene_list if g]
     if not cleaned_gene_list or len(cleaned_gene_list) < 3:
         # print(f"⚠️ Skip enrichment '{description}': Needs >= 3 non-empty genes (found {len(cleaned_gene_list)})."); # Printed in calling loop
         return pd.DataFrame() # Return empty DataFrame

     print(f"--- 📊 Performing Enrichment: {description} ({len(cleaned_gene_list)} genes) using '{gene_sets}' ---")
     try:
          libs = gp.get_library_name(organism='Human')
          valid_libs = [gene_sets] if gene_sets in libs else [] # Only use the specified library if valid
          if not valid_libs:
              print(f"❌ Invalid or unavailable gene_set: '{gene_sets}'. Available examples: {libs[:5]}")
              return pd.DataFrame()

          enr = gp.enrichr(gene_list=cleaned_gene_list, gene_sets=valid_libs, organism='Human', outdir=None, cutoff=0.05) # Use gp alias

          if enr is None or not hasattr(enr, 'results') or enr.results.empty:
               # print(f"ℹ️ No significant terms found for '{description}' using library: '{gene_sets}'."); # Printed in calling loop
               return pd.DataFrame() # Return empty DataFrame for consistency

          # Process results
          results_df = enr.results.sort_values('Adjusted P-value').reset_index(drop=True)
          # print(f"🧬 Top {min(top_n, len(results_df))} Enriched Terms ('{gene_sets}'):") # Printed in calling loop
          # Format for display - only if needed, not for return value
          # display_df=results_df[['Term', 'Adjusted P-value', 'Overlap', 'Genes']].head(top_n).copy()
          # display_df['Adjusted P-value'] = display_df['Adjusted P-value'].map('{:.2e}'.format)
          # display_df['Genes'] = display_df['Genes'].fillna('').astype(str).str.split(';').str[:5].apply(lambda x: ';'.join(x)+('...' if len(x)==5 else ''))
          # print(display_df.to_string(index=False)); print("-" * 70);

          return results_df # Return the full DataFrame
     except requests.exceptions.RequestException as e:
          print(f"❌ GSEAPY Enrichr Connection Error for '{description}': {e}");
          return None # Indicate error
     except Exception as e:
          print(f"❌ Unexpected enrichment error for '{description}': {e}");
          traceback.print_exc(limit=2);
          return None # Indicate error


# --- Helper Function: Plot Enrichment Results ---
# COPIED from ablation_08 Cell 2
# MODIFIED: Use GLOBALS for top_n and output_dir defaults.
# MODIFIED: Removed automatic saving and displaying; calling cell will handle that.
def create_enrichment_plot_simple(results_df, title):
    """
    Generates a horizontal bar plot for enrichment results.
    Returns the matplotlib Figure object or None. Does NOT save or display automatically.
    """
    # Access globals for configuration
    top_n = globals().get('TOP_N_ENRICHMENT_TERMS', 15)

    if not isinstance(results_df, pd.DataFrame) or results_df.empty: print(f"⚠️ Plot function skipped for '{title}': No data."); return None
    pval_col = 'Adjusted P-value'; term_col = 'Term'
    if pval_col not in results_df.columns or term_col not in results_df.columns: print(f"❌ Plot Error for '{title}': Missing columns '{pval_col}' or '{term_col}'."); return None

    print(f"--- Generating Enrichment Plot: {title} ---")
    plot_data = results_df.dropna(subset=[pval_col]).sort_values(by=pval_col, ascending=True).head(top_n).copy()
    if plot_data.empty: print("ℹ️ No terms left to plot after filtering/sorting."); return None
    epsilon = np.finfo(float).tiny
    plot_data['-log10 Adj P'] = -np.log10(plot_data[pval_col].replace(0, epsilon) + epsilon)
    plot_data.sort_values('-log10 Adj P', ascending=True, inplace=True)

    num_terms_plotted = len(plot_data); fig_height = max(6, num_terms_plotted * 0.5)
    fig, ax = plt.subplots(figsize=(10, fig_height))
    sns.barplot(x='-log10 Adj P', y=term_col, data=plot_data, palette='viridis_r', ax=ax, orient='h')
    ax.set_title(title, fontsize=13); ax.set_xlabel(f'-log10 ({pval_col})', fontsize=11); ax.set_ylabel('');
    ax.tick_params(axis='y', labelsize=10); ax.tick_params(axis='x', labelsize=10)
    ax.xaxis.grid(True, linestyle='--', alpha=0.6); ax.yaxis.grid(False)
    xlim_max = ax.get_xlim()[1]; text_offset = xlim_max * 0.01;
    for i, bar_val in enumerate(plot_data['-log10 Adj P']): ax.text(bar_val + text_offset , i, f'{bar_val:.2f}', va='center', ha='left', fontsize=9)
    plt.tight_layout()

    # Return the figure handle
    return fig


print("\n ✅ Cell 2: Canonical biological analysis helper functions defined (Using GLOBALS for config).")


--- Cell 2: Defining Canonical Helper Functions for Biological Analysis (2025-04-28 21:18:27) ---

 ✅ Cell 2: Canonical biological analysis helper functions defined (Using GLOBALS for config).


In [4]:
# Cell 3: Define Run Folders
# Description: Defines the mapping from human-readable labels to the actual
#              experiment folder names for ALL relevant runs (01-07).
#              MODIFIED: Sets run_folder_map as a global variable.
#              COPIED from ablation_09 Cell 2.

import os
import time

print(f"\n--- Cell 3: Define Run Folders ({time.strftime('%Y-%m-%d %H:%M:%S')}) ---")

# Ensure BASE_EXPERIMENT_NAME is defined globally (should be from Cell 1)
if 'BASE_EXPERIMENT_NAME' not in globals() or not BASE_EXPERIMENT_NAME:
    print("❌ Error: BASE_EXPERIMENT_NAME not defined globally. Run Cell 1.")
    # No need to set run_folder_map as global if error
    run_folder_map = {} # Define locally as empty dict
else:
    # Define the mapping from analysis label to the actual folder name
    # These suffixes must match the EXPERIMENT_NAME set in Cell 1 of each run notebook (ablation_01-07)
    run_folder_map = {
        "H+P (2D Ref)": f"{BASE_EXPERIMENT_NAME}_LinearHarmonicPheromone_REF", # from ablation_01
        "P-Only (2D)": f"{BASE_EXPERIMENT_NAME}_LinearPheromoneOnly",         # from ablation_02
        "H-Only (2D)": f"{BASE_EXPERIMENT_NAME}_LinearHarmonicOnly",           # from ablation_03
        "H+3D-PH (Coupled)": f"{BASE_EXPERIMENT_NAME}_LinearHarmonic_PlaceholderDim3D", # from ablation_04
        "H+5D-PH (Coupled)": f"{BASE_EXPERIMENT_NAME}_LinearHarmonic_PlaceholderDim5D", # from ablation_05
        "H+5D-PH (Decoupled)": f"{BASE_EXPERIMENT_NAME}_LinearHarmonic_PlaceholderDim5D_DecoupledDiff", # from ablation_06
        "H+4D-Bio (AIFM1)": f"{BASE_EXPERIMENT_NAME}_Harmonic4DBio"           # from ablation_07
    }

    # --- MODIFIED: Set run_folder_map as a global variable ---
    globals()['run_folder_map'] = run_folder_map
    # --- END MODIFIED ---

    print("Defined mapping of run labels to expected folder names for dynamic analysis:")
    for label, folder in run_folder_map.items():
        print(f"  '{label}': '{folder}'")

print("\n ✅ Cell 3: Run folders defined.")


--- Cell 3: Define Run Folders (2025-04-28 21:18:27) ---
Defined mapping of run labels to expected folder names for dynamic analysis:
  'H+P (2D Ref)': 'string_ca_subgraph_AIFM1_CORRECTED_LinearHarmonicPheromone_REF'
  'P-Only (2D)': 'string_ca_subgraph_AIFM1_CORRECTED_LinearPheromoneOnly'
  'H-Only (2D)': 'string_ca_subgraph_AIFM1_CORRECTED_LinearHarmonicOnly'
  'H+3D-PH (Coupled)': 'string_ca_subgraph_AIFM1_CORRECTED_LinearHarmonic_PlaceholderDim3D'
  'H+5D-PH (Coupled)': 'string_ca_subgraph_AIFM1_CORRECTED_LinearHarmonic_PlaceholderDim5D'
  'H+5D-PH (Decoupled)': 'string_ca_subgraph_AIFM1_CORRECTED_LinearHarmonic_PlaceholderDim5D_DecoupledDiff'
  'H+4D-Bio (AIFM1)': 'string_ca_subgraph_AIFM1_CORRECTED_Harmonic4DBio'

 ✅ Cell 3: Run folders defined.


In [5]:
# Cell 4: Calculate Dynamic Region for Each Run (Loads Histories)
# Description: Iterates through the defined run folders and loads the Activation
#              and Inhibition history DataFrames, calculates the dynamic region metric,
#              and identifies the dynamic nodes based on the percentile threshold.
#              Stores the results.
#              Requires run folder map (Cell 3) and helper functions (Cell 1.1).
#              COPIED from ablation_09 Cell 4.
#              MODIFIED: Corrected the directory path used to load the history files.
#              FIXED: Ensure run_dynamic_sizes, run_dynamic_metrics, run_dynamic_thresholds,
#                     and run_dynamic_nodes_lists are stored globally *at the end of Cell 4*
#                     so they are available for Cell 7 and other subsequent cells.
#              FIXED: Ensure the *list of dynamic region node IDs* for each run is stored
#                     in the `loaded_dynamic_regions` dictionary, as expected by Cell 5.

import os
import time
import pandas as pd
import numpy as np # Needed for np.nan, np.percentile, np.where, np.nan_to_num
# Helper functions load_history_dfs, calculate_time_avg_abs_change_metric, identify_dynamic_nodes defined in Cell 1.1

print(f"\n--- Cell 4: Calculate Dynamic Region for Each Run (Loads Histories) ({time.strftime('%Y-%m-%d %H:%M:%S')}) ---")

# --- Prerequisites Check ---
dynamic_region_calc_error = False

# Check run folder map (from Cell 3)
if 'run_folder_map' not in globals() or not isinstance(run_folder_map, dict) or not run_folder_map:
    print("❌ Dynamic Region Calc Error: 'run_folder_map' is missing or invalid (Run Cell 3)."); dynamic_region_calc_error = True

# --- Check dynamic analysis output directory (from Cell 1) ---
# This should be the output directory of ablation_09
if 'INPUT_DIR_DYNAMIC_REGIONS' not in globals() or not INPUT_DIR_DYNAMIC_REGIONS:
     print("❌ Dynamic Region Calc Error: Dynamic analysis input directory global 'INPUT_DIR_DYNAMIC_REGIONS' missing (Run Cell 1)."); dynamic_region_calc_error = True
elif not os.path.isdir(INPUT_DIR_DYNAMIC_REGIONS):
     print(f"❌ Dynamic Region Calc Error: Dynamic analysis input directory not found: {INPUT_DIR_DYNAMIC_REGIONS}. Ensure ablation_09 was run and created this directory."); dynamic_region_calc_error = True
# --- END Corrected Check ---


# Check history loading helper (Cell 1.1)
if 'load_history_dfs' not in globals() or not callable(load_history_dfs):
    print("❌ Dynamic Region Calc Error: History loading function 'load_history_dfs' missing (Defined in Cell 1.1?)."); dynamic_region_calc_error = True
# Check metric calculation helper (Cell 1.1)
if 'calculate_time_avg_abs_change_metric' not in globals() or not callable(calculate_time_avg_abs_change_metric):
    print("❌ Dynamic Region Calc Error: Metric calculation function missing (Defined in Cell 1.1?)."); dynamic_region_calc_error = True
# Check node identification helper (Cell 1.1)
if 'identify_dynamic_nodes' not in globals() or not callable(identify_dynamic_nodes):
    print("❌ Dynamic Region Calc Error: Node identification function missing (Defined in Cell 1.1?)."); dynamic_region_calc_error = True
# Check dynamic region parameters (Cell 1 - accessed via GLOBALS by helpers)
if 'DYNAMIC_WINDOW_FRACTION' not in globals(): print("❌ Dynamic Region Calc Error: Required global 'DYNAMIC_WINDOW_FRACTION' missing (Run Cell 1)."); dynamic_region_calc_error = True
if 'DYNAMIC_THRESHOLD_VALUE' not in globals(): print("❌ Dynamic Region Calc Error: Required global 'DYNAMIC_THRESHOLD_VALUE' missing (Run Cell 1)."); dynamic_region_calc_error = True
if 'DYNAMIC_THRESHOLD_TYPE' not in globals(): print("❌ Dynamic Region Calc Error: Required global 'DYNAMIC_THRESHOLD_TYPE' missing (Run Cell 1)."); dynamic_region_calc_error = True
if 'DYNAMIC_METRIC_KEY' not in globals(): print("❌ Dynamic Region Calc Error: Required global 'DYNAMIC_METRIC_KEY' missing (Run Cell 1)."); dynamic_region_calc_error = True # Needed by metric func


# Check output directory for saving dynamic region node lists (from Cell 1)
if 'OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS' not in globals() or not OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS:
     print("❌ Dynamic Region Calc Error: Output directory global 'OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS' missing (Run Cell 1)."); dynamic_region_calc_error = True
elif not os.path.isdir(OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS):
     print(f"❌ Dynamic Region Calc Error: Output directory not found: {OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS}. Check Cell 1."); dynamic_region_calc_error = True


# --- Initialize dictionaries to store results ---
# MODIFIED: Initialize the dictionaries that will be stored globally
run_dynamic_metrics = {} # {run_label: pandas Series of metric values per node}
run_dynamic_nodes_lists = {} # {run_label: list of dynamic node IDs}
run_dynamic_thresholds = {} # {run_label: actual threshold value used}
run_dynamic_sizes = {} # {run_label: number of dynamic nodes}
# --- Added: Initialize the dictionary expected by Cell 5 ---
loaded_dynamic_regions = {} # {run_label: list of dynamic node IDs} - This is what Cell 5 expects
# END MODIFIED


# --- Execute Dynamic Region Calculation ---
if not dynamic_region_calc_error:
    print("Calculating dynamic region metric and identifying dynamic nodes for each run...")

    for label, folder_name in run_folder_map.items():
        print(f"  Processing '{label}' (Folder: {folder_name})...")

        # --- Load history for this run ---
        # MODIFIED: Load history from the correct directory (OUTPUT_DIR_SIMULATIONS)
        act_df, inh_df = load_history_dfs(folder_name, OUTPUT_DIR_SIMULATIONS)
        # END MODIFIED

        if act_df is None or inh_df is None:
            print(f"    Skipping '{label}': History DataFrames not available or invalid.")
            run_dynamic_metrics[label] = None
            run_dynamic_nodes_lists[label] = []
            loaded_dynamic_regions[label] = [] # Store empty list for Cell 5
            run_dynamic_thresholds[label] = np.nan # Use NaN for numeric threshold
            run_dynamic_sizes[label] = 0
            continue

        # --- Calculate the Dynamic Metric for this run ---
        # The helper accesses parameters from GLOBALS
        node_metric_values = calculate_time_avg_abs_change_metric(act_df, inh_df)

        if node_metric_values is None:
            print(f"    Skipping '{label}': Metric calculation failed or resulted in None.")
            run_dynamic_metrics[label] = None
            run_dynamic_nodes_lists[label] = []
            loaded_dynamic_regions[label] = [] # Store empty list for Cell 5
            run_dynamic_thresholds[label] = np.nan
            run_dynamic_sizes[label] = 0
            continue
        if node_metric_values.empty:
             print(f"    Skipping '{label}': Metric calculation resulted in an empty Series.")
             run_dynamic_metrics[label] = node_metric_values # Store empty series
             run_dynamic_nodes_lists[label] = []
             loaded_dynamic_regions[label] = [] # Store empty list for Cell 5
             run_dynamic_thresholds[label] = np.nan
             run_dynamic_sizes[label] = 0
             continue


        run_dynamic_metrics[label] = node_metric_values # Store the metric values Series

        # --- Identify Dynamic Nodes for this run ---
        # The helper accesses threshold parameters from GLOBALS
        dynamic_nodes_list = identify_dynamic_nodes(node_metric_values)

        run_dynamic_nodes_lists[label] = dynamic_nodes_list # Store the list of node IDs
        # --- Added: Store the list in the variable expected by Cell 5 ---
        loaded_dynamic_regions[label] = dynamic_nodes_list
        # --- End Added ---
        run_dynamic_sizes[label] = len(dynamic_nodes_list) # Store the size

        # --- Store the *actual* threshold used for this run ---
        # Need to recalculate it here because identify_dynamic_nodes returns just the list, not the threshold
        actual_threshold_for_run = np.nan # Initialize
        valid_metrics = node_metric_values.dropna().values # Exclude NaNs for threshold calc
        if len(valid_metrics) > 0:
             threshold_type = globals().get('DYNAMIC_THRESHOLD_TYPE', 'percentile')
             threshold_value = globals().get('DYNAMIC_THRESHOLD_VALUE', 80)
             try:
                  if threshold_type == 'percentile':
                       # Ensure value is within bounds for percentile
                       percentile_val = threshold_value
                       if not (0 <= percentile_val <= 100): percentile_val = 80 # Default if invalid

                       actual_threshold_for_run = np.percentile(valid_metrics, percentile_val)
                  elif threshold_type == 'absolute':
                       actual_threshold_for_run = threshold_value
                  else: warnings.warn(f"    Unknown threshold type in globals: '{threshold_type}'. Threshold will be NaN.")
             except Exception as e_thresh_calc: warnings.warn(f"    Error calculating threshold for '{label}': {e_thresh_calc}. Threshold will be NaN.");

        run_dynamic_thresholds[label] = actual_threshold_for_run # Store the calculated/used threshold

        print(f"    ✅ Identified {run_dynamic_sizes[label]} dynamic nodes (Threshold: {actual_threshold_for_run:.6f} {globals().get('DYNAMIC_THRESHOLD_TYPE','')} )")

        # --- Save Dynamic Region Nodes to File ---
        if dynamic_nodes_list:
             # Save to the correct output directory for this notebook (OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS)
             dynamic_region_filename = os.path.join(OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS, f"dynamic_region_nodes_{label}.txt") # Use label in filename
             try:
                  with open(dynamic_region_filename, 'w') as f:
                       for node_id in dynamic_nodes_list: f.write(f"{node_id}\n")
                  print(f"    ✅ Saved dynamic region nodes to: {dynamic_region_filename}")
             except Exception as e: print(f"    ❌ Error saving dynamic region node list: {e}")


    print("\nFinished processing dynamic regions for all runs.")

else: # dynamic_region_calc_error was True
    print("Skipping dynamic region calculation due to missing prerequisites.")


# --- Store globally at the end of Cell 4 ---
globals()['run_dynamic_metrics'] = run_dynamic_metrics # The metric values per node
globals()['run_dynamic_nodes_lists'] = run_dynamic_nodes_lists # The lists of identified nodes
globals()['run_dynamic_thresholds'] = run_dynamic_thresholds # The actual thresholds used
globals()['run_dynamic_sizes'] = run_dynamic_sizes # The size of the dynamic region
# --- Added: Store the variable expected by Cell 5 ---
globals()['loaded_dynamic_regions'] = loaded_dynamic_regions
# --- End Added ---


print("\nCell 4: Dynamic Region calculation complete.")


--- Cell 4: Calculate Dynamic Region for Each Run (Loads Histories) (2025-04-28 21:18:27) ---
Calculating dynamic region metric and identifying dynamic nodes for each run...
  Processing 'H+P (2D Ref)' (Folder: string_ca_subgraph_AIFM1_CORRECTED_LinearHarmonicPheromone_REF)...
    ✅ Identified 467 dynamic nodes (Threshold: 1.331264 percentile )
    ✅ Saved dynamic region nodes to: biological_analysis_results/Dynamic_Biological_Analysis/dynamic_region_nodes_H+P (2D Ref).txt
  Processing 'P-Only (2D)' (Folder: string_ca_subgraph_AIFM1_CORRECTED_LinearPheromoneOnly)...
    ✅ Identified 467 dynamic nodes (Threshold: 0.000735 percentile )
    ✅ Saved dynamic region nodes to: biological_analysis_results/Dynamic_Biological_Analysis/dynamic_region_nodes_P-Only (2D).txt
  Processing 'H-Only (2D)' (Folder: string_ca_subgraph_AIFM1_CORRECTED_LinearHarmonicOnly)...
    ✅ Identified 467 dynamic nodes (Threshold: 1.321709 percentile )
    ✅ Saved dynamic region nodes to: biological_analysis_results

In [6]:
# Cell 5: Map Dynamic Region Nodes to Gene Symbols
# Description: Iterates through the loaded dynamic region node lists, maps the STRING IDs
#              to gene symbols for each run's list using the helper function.
#              Stores the resulting lists of gene symbols.
#              Requires dynamic region lists (Cell 4) and helper function (Cell 2).

import os
import time
import pandas as pd # Keep pandas import if needed later
# Helper function map_string_ids_to_gene_symbols_api defined in Cell 2

print(f"\n--- Cell 5: Map Dynamic Region Nodes to Gene Symbols ({time.strftime('%Y-%m-%d %H:%M:%S')}) ---")

# --- Prerequisites Check ---
mapping_error = False
# Check loaded dynamic regions (from Cell 4)
if 'loaded_dynamic_regions' not in globals() or not isinstance(loaded_dynamic_regions, dict):
    print("❌ Mapping Error: 'loaded_dynamic_regions' missing or invalid (Run Cell 4)."); mapping_error = True
elif not any(loaded_dynamic_regions.values()): # Check if *any* of the lists are non-empty
    print("⚠️ Skipping Mapping: All dynamic region node lists loaded in Cell 4 are empty."); mapping_error = True
# Check mapping helper function (Cell 2)
if 'map_string_ids_to_gene_symbols_api' not in globals() or not callable(map_string_ids_to_gene_symbols_api):
    print("❌ Mapping Error: Helper function 'map_string_ids_to_gene_symbols_api' missing (Defined in Cell 2?)."); mapping_error = True
# Check necessary config parameters (Cell 1 - accessed via GLOBALS by helper)
if 'SPECIES_ID' not in globals(): print("❌ Mapping Error: Required global 'SPECIES_ID' missing (Run Cell 1)."); mapping_error = True
if 'TARGET_NODE_NAME' not in globals(): print("⚠️ Mapping Warning: Required global 'TARGET_NODE_NAME' missing (Run Cell 1). Description may be generic."); # Allow proceed
if 'OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS' not in globals() or not OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS:
     print("❌ Mapping Error: Required global 'OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS' missing (Run Cell 1)."); mapping_error = True
elif not os.path.isdir(OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS):
     print(f"❌ Mapping Error: Output directory not found: {OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS}. Check Cell 1."); mapping_error = True


# --- Initialize dictionary to store mapped gene symbols ---
# {run_label: list of gene symbols}
mapped_dynamic_regions_genes = {}
genes_mapped_available = False # Flag if any lists were successfully mapped and non-empty

# --- Execute Mapping ---
if not mapping_error:
    print(f"Attempting to map dynamic region node IDs to gene symbols...")

    for label, node_list in loaded_dynamic_regions.items():
        if not node_list:
            print(f"  Skipping mapping for '{label}': Node list is empty.")
            mapped_dynamic_regions_genes[label] = [] # Store empty list
            continue

        print(f"  Mapping {len(node_list)} nodes for '{label}'...")
        try:
            # Call the helper function defined in Cell 2
            # It accesses config parameters from GLOBALS and uses the output dir of this notebook for cache
            gene_symbols_list = map_string_ids_to_gene_symbols_api(
                string_ids=node_list,
                description=f"Dynamic Region '{label}'" # Use label in description
            )

            mapped_dynamic_regions_genes[label] = gene_symbols_list # Store the list of gene symbols

            if gene_symbols_list:
                 print(f"    ✅ Mapped to {len(gene_symbols_list)} gene symbols for '{label}'.")
                 genes_mapped_available = True # Set flag if at least one list is non-empty
            else:
                 print(f"    ⚠️ Mapping yielded no gene symbols for '{label}'.")

        except Exception as e:
            print(f"  ❌ An error occurred during mapping for '{label}': {e}")
            traceback.print_exc()
            mapped_dynamic_regions_genes[label] = [] # Store empty list on error
            # Do not set fatal mapping_error, allow others to run


    print("\nFinished attempting to map gene symbols for dynamic regions.")
    if not genes_mapped_available:
         warnings.warn("⚠️ No dynamic region lists were successfully mapped to gene symbols. Biological analysis will be skipped.")


else:
    print("Skipping gene symbol mapping due to previous errors or all input lists being empty")

# Store globally for subsequent cells
globals()['mapped_dynamic_regions_genes'] = mapped_dynamic_regions_genes

print("\nCell 5: Dynamic Region node mapping complete.")



--- Cell 5: Map Dynamic Region Nodes to Gene Symbols (2025-04-28 21:18:30) ---
Attempting to map dynamic region node IDs to gene symbols...
  Mapping 467 nodes for 'H+P (2D Ref)'...
Mapping 467 Dynamic Region 'H+P (2D Ref)' IDs...
  🌐 Mapping 467 IDs via STRING API...


API Mapping (Dynamic Region 'H+P (2D Ref)'):   0%|          | 0/5 [00:00<?, ?it/s]

  ✅ Updated mapping cache saved (467 total mappings).
    ✅ Mapped to 467 gene symbols for 'H+P (2D Ref)'.
  Mapping 467 nodes for 'P-Only (2D)'...
Mapping 467 Dynamic Region 'P-Only (2D)' IDs...
  ℹ️ Loaded 467 mappings from cache: biological_analysis_results/Dynamic_Biological_Analysis/string_id_to_gene_symbol_map_cache.pkl
  🌐 Mapping 372 IDs via STRING API...


API Mapping (Dynamic Region 'P-Only (2D)'):   0%|          | 0/4 [00:00<?, ?it/s]

  ✅ Updated mapping cache saved (839 total mappings).
    ✅ Mapped to 467 gene symbols for 'P-Only (2D)'.
  Mapping 467 nodes for 'H-Only (2D)'...
Mapping 467 Dynamic Region 'H-Only (2D)' IDs...
  ℹ️ Loaded 839 mappings from cache: biological_analysis_results/Dynamic_Biological_Analysis/string_id_to_gene_symbol_map_cache.pkl
  🌐 Mapping 144 IDs via STRING API...


API Mapping (Dynamic Region 'H-Only (2D)'):   0%|          | 0/2 [00:00<?, ?it/s]

  ✅ Updated mapping cache saved (983 total mappings).
    ✅ Mapped to 467 gene symbols for 'H-Only (2D)'.
  Mapping 467 nodes for 'H+3D-PH (Coupled)'...
Mapping 467 Dynamic Region 'H+3D-PH (Coupled)' IDs...
  ℹ️ Loaded 983 mappings from cache: biological_analysis_results/Dynamic_Biological_Analysis/string_id_to_gene_symbol_map_cache.pkl
  🌐 Mapping 96 IDs via STRING API...
  ✅ Updated mapping cache saved (1079 total mappings).
    ✅ Mapped to 467 gene symbols for 'H+3D-PH (Coupled)'.
  Mapping 467 nodes for 'H+5D-PH (Coupled)'...
Mapping 467 Dynamic Region 'H+5D-PH (Coupled)' IDs...
  ℹ️ Loaded 1079 mappings from cache: biological_analysis_results/Dynamic_Biological_Analysis/string_id_to_gene_symbol_map_cache.pkl
  🌐 Mapping 39 IDs via STRING API...
  ✅ Updated mapping cache saved (1118 total mappings).
    ✅ Mapped to 467 gene symbols for 'H+5D-PH (Coupled)'.
  Mapping 467 nodes for 'H+5D-PH (Decoupled)'...
Mapping 467 Dynamic Region 'H+5D-PH (Decoupled)' IDs...
  ℹ️ Loaded 1118 mappi

In [7]:
# Cell 6: Perform Enrichment Analysis for Each Dynamic Region
# Description: Iterates through the mapped gene symbol lists and performs GO BP
#              enrichment analysis for each run's dynamic region. Stores the results.
#              Requires mapped gene lists (Cell 5) and helper function (Cell 2).

import os
import time
import pandas as pd
# Helper function run_enrichment_simple defined in Cell 2

# Ensure gseapy is available (checked in Cell 0 or earlier)
try: import gseapy as gp # Use alias
except ImportError: gp = None

print(f"\n--- Cell 6: Perform Enrichment Analysis for Each Dynamic Region ({time.strftime('%Y-%m-%d %H:%M:%S')}) ---")

# --- Prerequisites Check ---
enrichment_error = False
# Check mapped gene lists (from Cell 5)
if 'mapped_dynamic_regions_genes' not in globals() or not isinstance(mapped_dynamic_regions_genes, dict):
    print("❌ Enrichment Error: 'mapped_dynamic_regions_genes' missing or invalid (Run Cell 5)."); enrichment_error = True
elif not any(mapped_dynamic_regions_genes.values()): # Check if *any* of the lists are non-empty
    print("⚠️ Skipping Enrichment: All mapped gene symbol lists are empty."); enrichment_error = True
# Check enrichment helper function (Cell 2)
if 'run_enrichment_simple' not in globals() or not callable(run_enrichment_simple):
    print("❌ Enrichment Error: Helper function 'run_enrichment_simple' missing (Defined in Cell 2?)."); enrichment_error = True
# Check gseapy availability (handled by import check and gp alias)
if gp is None: print("❌ Enrichment Error: gseapy library not available."); enrichment_error = True
# Check necessary config parameters (Cell 1 - accessed via GLOBALS by helper)
if 'GO_ENRICHMENT_LIBRARY' not in globals(): print("❌ Enrichment Error: Required global 'GO_ENRICHMENT_LIBRARY' missing (Run Cell 1)."); enrichment_error = True
if 'TOP_N_ENRICHMENT_TERMS' not in globals(): print("⚠️ Enrichment Warning: Required global 'TOP_N_ENRICHMENT_TERMS' missing (Run Cell 1). Display may be default."); # Allow proceed
if 'OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS' not in globals() or not OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS:
     print("❌ Enrichment Error: Required global 'OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS' missing (Run Cell 1)."); enrichment_error = True
elif not os.path.isdir(OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS):
     print(f"❌ Enrichment Error: Output directory not found: {OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS}. Check Cell 1."); enrichment_error = True


# --- Initialize dictionary to store enrichment results ---
# {run_label: pandas DataFrame of enrichment results (can be empty if no sig terms)}
dynamic_regions_enrichment_results = {}

# --- Execute Enrichment ---
if not enrichment_error:
    print(f"Attempting to perform GO enrichment for dynamic regions...")

    for label, gene_list in mapped_dynamic_regions_genes.items():
        if not gene_list or len(gene_list) < 3:
            print(f"  Skipping enrichment for '{label}': Fewer than 3 genes ({len(gene_list)}).")
            dynamic_regions_enrichment_results[label] = pd.DataFrame() # Store empty DF
            continue

        print(f"  Running enrichment for '{label}' ({len(gene_list)} genes)...")
        try:
            # Call the helper function defined in Cell 2
            # It accesses config parameters (library, top_n) from GLOBALS
            results_df = run_enrichment_simple(
                gene_list=gene_list,
                description=f"Dynamic Region '{label}'" # Use label in description
            )

            # Store the results DataFrame (can be empty if no significant terms)
            if isinstance(results_df, pd.DataFrame):
                 dynamic_regions_enrichment_results[label] = results_df
                 if results_df.empty:
                      print(f"    ℹ️ No significant terms found for '{label}'.")
                 else:
                      print(f"    ✅ Found {len(results_df)} significant terms for '{label}'.")
                      # Optional: Save full results for this run
                      enr_filename = os.path.join(OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS, f"dynamic_region_{label}_enrichment_results.csv")
                      try:
                           results_df.to_csv(enr_filename, index=False)
                      except Exception as e_save: print(f"    ❌ Error saving enrichment results CSV for '{label}': {e_save}")

            elif results_df is None:
                 print(f"    ❌ Enrichment function returned None for '{label}'.")
                 dynamic_regions_enrichment_results[label] = pd.DataFrame() # Store empty DF on None return

        except Exception as e:
            print(f"  ❌ An unexpected error occurred during enrichment for '{label}': {e}")
            traceback.print_exc()
            dynamic_regions_enrichment_results[label] = pd



--- Cell 6: Perform Enrichment Analysis for Each Dynamic Region (2025-04-28 21:18:52) ---
Attempting to perform GO enrichment for dynamic regions...
  Running enrichment for 'H+P (2D Ref)' (467 genes)...
--- 📊 Performing Enrichment: Dynamic Region 'H+P (2D Ref)' (467 genes) using 'GO_Biological_Process_2023' ---
    ✅ Found 2612 significant terms for 'H+P (2D Ref)'.
  Running enrichment for 'P-Only (2D)' (467 genes)...
--- 📊 Performing Enrichment: Dynamic Region 'P-Only (2D)' (467 genes) using 'GO_Biological_Process_2023' ---
    ✅ Found 2544 significant terms for 'P-Only (2D)'.
  Running enrichment for 'H-Only (2D)' (467 genes)...
--- 📊 Performing Enrichment: Dynamic Region 'H-Only (2D)' (467 genes) using 'GO_Biological_Process_2023' ---
    ✅ Found 2560 significant terms for 'H-Only (2D)'.
  Running enrichment for 'H+3D-PH (Coupled)' (467 genes)...
--- 📊 Performing Enrichment: Dynamic Region 'H+3D-PH (Coupled)' (467 genes) using 'GO_Biological_Process_2023' ---
    ✅ Found 2512 sign

In [8]:
# Cell 7: Compile and Format Comparative Enrichment Table
# Description: Gathers key enrichment metrics (number of terms, top term, p-value)
#              for the dynamic region of each run and compiles them into a pandas DataFrame.
#              Formats the DataFrame for presentation in the final markdown summary.
#              Requires dynamic region enrichment results (Cell 6) and the dynamic analysis table (from ablation_09).
#              MODIFIED: Compiles the main comparative *enrichment* table by merging data from Cell 6 and the ablation_09 output file.
#              MODIFIED: Added explicit saving of the resulting DataFrame to a CSV file.

import os
import time
import pandas as pd
import numpy as np
import warnings
import traceback

print(f"\n--- Cell 7: Compile and Format Comparative Enrichment Table ({time.strftime('%Y-%m-%d %H:%M:%S')}) ---")

# --- Prerequisites Check ---
table_creation_error = False

# Check enrichment results DataFrame from Cell 6 of THIS notebook
if 'dynamic_regions_enrichment_results' not in globals() or not isinstance(dynamic_regions_enrichment_results, dict):
    print("❌ Table Creation Error: 'dynamic_regions_enrichment_results' missing or invalid (Run Cell 6)."); table_creation_error = True
# Note: It's okay if the dict is empty or contains empty DataFrames; the table will reflect that.

# Check the dynamic analysis comparison table (from ablation_09 output file)
# We will *load* this file here, so check the file path global (from Cell 1)
if 'OUTPUT_DIR_DYNAMIC_ANALYSIS' not in globals() or not OUTPUT_DIR_DYNAMIC_ANALYSIS:
     print("❌ Table Creation Error: Dynamic analysis output directory global 'OUTPUT_DIR_DYNAMIC_ANALYSIS' missing (Run Cell 1). Cannot load dynamic analysis table."); table_creation_error = True
else:
     # Attempt to load the dynamic analysis comparison table CSV
     dynamic_analysis_table_path = os.path.join(OUTPUT_DIR_DYNAMIC_ANALYSIS, "dynamic_analysis_comparison_table.csv") # Assumed filename from ablation_09
     dynamic_analysis_df_loaded = pd.DataFrame() # Initialize empty
     try:
          if os.path.exists(dynamic_analysis_table_path):
               dynamic_analysis_df_loaded = pd.read_csv(dynamic_analysis_table_path, index_col=0)
               print(f"  ✅ Loaded dynamic analysis comparison table from: {dynamic_analysis_table_path}")
               if dynamic_analysis_df_loaded.empty: warnings.warn("  ⚠️ Loaded dynamic analysis comparison table is empty.")
          else:
               print(f"❌ Table Creation Error: Dynamic analysis comparison table not found at: {dynamic_analysis_table_path}. Ensure ablation_09 was run and completed successfully.");
               table_creation_error = True # Critical error if this file is missing

     except Exception as e_load_dynamic_analysis:
          print(f"❌ Error loading dynamic analysis comparison table: {e_load_dynamic_analysis}"); traceback.print_exc();
          table_creation_error = True # Critical error

# Ensure we have valid data to merge, even if one source is empty
if not table_creation_error and not (dynamic_regions_enrichment_results or not dynamic_analysis_df_loaded.empty):
     print("⚠️ Skipping Table Creation: No run data available from either enrichment results (Cell 6) or dynamic analysis table (ablation_09 output).")
     table_creation_error = True # No data to process


# --- Compile Data for the Comparative Enrichment Table ---
# This table will combine summary enrichment metrics with dynamic analysis metrics for each run
enrichment_comparison_table_data = {} # {run_label: {metric_name: value}}

if not table_creation_error:
    print("Compiling data for comparative enrichment table...")

    # Get all run labels from the enrichment results dictionary AND the dynamic analysis table
    # This ensures all runs are included if they appeared in either source
    all_run_labels = sorted(list(dynamic_regions_enrichment_results.keys()) + list(dynamic_analysis_df_loaded.index))
    all_run_labels = sorted(list(set(all_run_labels))) # Get unique and sort

    if not all_run_labels:
        print("⚠️ No run labels found to compile data for the enrichment table.")
        table_creation_error = True # Nothing to put in the table

if not table_creation_error:
    for label in all_run_labels:
        enrichment_comparison_table_data[label] = {} # Initialize entry for this run

        # --- Add Enrichment Metrics (from Cell 6 results) ---
        enrichment_df = dynamic_regions_enrichment_results.get(label)

        if isinstance(enrichment_df, pd.DataFrame):
            if not enrichment_df.empty:
                # Enrichment ran and found significant terms
                num_terms = len(enrichment_df)

                # Safely get Top Term and Top Term Adj P from the DataFrame
                top_term = enrichment_df.iloc[0].get('Term', 'N/A')
                top_pval = enrichment_df.iloc[0].get('Adjusted P-value', np.nan) # Use get with NaN default

                enrichment_comparison_table_data[label]['Significant Terms'] = num_terms
                enrichment_comparison_table_data[label]['Top Term'] = top_term
                enrichment_comparison_table_data[label]['Top Term Adj P'] = top_pval

            else:
                # Enrichment ran but found no significant terms (empty DF)
                enrichment_comparison_table_data[label]['Significant Terms'] = 0
                enrichment_comparison_table_data[label]['Top Term'] = "No significant terms"
                enrichment_comparison_table_data[label]['Top Term Adj P'] = np.nan # Use NaN for p-value

        else: # Enrichment did not run or failed for this specific run (e.g., returned None)
            # Add placeholder values indicating the analysis was skipped or failed for this run
            enrichment_comparison_table_data[label]['Significant Terms'] = "Error/Skipped"
            enrichment_comparison_table_data[label]['Top Term'] = "Error/Skipped"
            enrichment_comparison_table_data[label]['Top Term Adj P'] = np.nan # Use NaN for p-value


        # --- Add Dynamic Analysis Metrics (from ablation_09 output) ---
        # Get row from the loaded dynamic analysis DataFrame, default to empty Series if label not in index
        dynamic_analysis_row = dynamic_analysis_df_loaded.loc[label] if label in dynamic_analysis_df_loaded.index else pd.Series(dtype=object) # Use object dtype for mixed types

        # Populate table entry with dynamic analysis metrics, defaulting to N/A or NaN if column/row is missing
        enrichment_comparison_table_data[label]['Dynamic Region Size'] = dynamic_analysis_row.get('Dynamic Region Size', np.nan)
        enrichment_comparison_table_data[label]['Mapped Genes'] = dynamic_analysis_row.get('Mapped Genes', np.nan) # Mapped Genes is calculated in THIS notebook
        enrichment_comparison_table_data[label]['Dynamic Metric Threshold'] = dynamic_analysis_row.get('Dynamic Metric Threshold', np.nan)
        enrichment_comparison_table_data[label][f'{globals().get("DYNAMIC_METRIC_KEY", "Metric")} (Avg in Region)'] = dynamic_analysis_row.get(f'{globals().get("DYNAMIC_METRIC_NAME", "Dynamic Metric")} (Avg in Region)', np.nan)

        # Add Jaccard columns
        jaccard_cols_prefix = 'Jaccard vs '
        # Find all Jaccard columns in the loaded dynamic analysis DataFrame
        jaccard_cols_in_loaded_df = [col for col in dynamic_analysis_df_loaded.columns if col.startswith(jaccard_cols_prefix)]
        for jaccard_col in jaccard_cols_in_loaded_df:
             enrichment_comparison_table_data[label][jaccard_col] = dynamic_analysis_row.get(jaccard_col, np.nan)


    # --- Create pandas DataFrame from compiled data ---
    # Define desired column order for the final table
    # Ensure dynamic analysis columns are included in the order
    column_order = [
        'Dynamic Region Size',
        'Mapped Genes', # Mapped Genes is from THIS notebook (Cell 5's output), added here
        'Significant Terms',
        'Top Term',
        'Top Term Adj P',
        'Dynamic Metric Threshold',
        f'{globals().get("DYNAMIC_METRIC_KEY", "Metric")} (Avg in Region)',
        # Add Jaccard columns dynamically based on the headers found
    ]
    # Add Jaccard columns found in the loaded dynamic analysis DF to the end of the order
    column_order.extend(sorted(jaccard_cols_in_loaded_df)) # Add and sort Jaccard columns

    # Create DataFrame directly from the compiled dictionary
    # This will automatically handle columns that might be missing for some rows (they'll be NaN)
    final_enrichment_comparison_df = pd.DataFrame.from_dict(enrichment_comparison_table_data, orient='index')

    # Reindex the DataFrame to enforce column order, dropping any columns not in the list
    existing_and_ordered_cols = [col for col in column_order if col in final_enrichment_comparison_df.columns]
    final_enrichment_comparison_df = final_enrichment_comparison_df[existing_and_ordered_cols].copy()


    # --- Format numeric columns for display ---
    # Identify numeric columns
    numeric_cols = final_enrichment_comparison_df.select_dtypes(include=np.number).columns.tolist()

    for col in numeric_cols:
        # Apply formatting, handling NaN values gracefully
        # Format as integer or float with N/A for NaN
        if col in ['Dynamic Region Size', 'Mapped Genes', 'Significant Terms']:
            # Format as integer, handling potential NaN/non-numeric
             final_enrichment_comparison_df[col] = final_enrichment_comparison_df[col].apply(lambda x: int(x) if pd.notna(x) and isinstance(x, (int, float)) else x)
        elif col == 'Top Term Adj P':
            # Apply formatting for p-value (exponential), handling NaN/non-numeric
             final_enrichment_comparison_df[col] = final_enrichment_comparison_df[col].apply(lambda x: f"{x:.2e}" if pd.notna(x) and isinstance(x, (int, float, np.number)) else "N/A")
        else: # Default formatting for other floats (threshold, average metric, Jaccard)
             final_enrichment_comparison_df[col] = final_enrichment_comparison_df[col].apply(lambda x: f"{x:.4f}" if pd.notna(x) and isinstance(x, (int, float, np.number)) else "N/A")


    print("✅ Comparative enrichment table DataFrame created and formatted.")


    # --- MODIFIED: Explicitly save the compiled DataFrame to CSV ---
    table_output_filename = os.path.join(OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS, "dynamic_biological_enrichment_comparison_table.csv")
    try:
        # Save with index=True to keep the run labels as the first column
        final_enrichment_comparison_df.to_csv(table_output_filename, index=True)
        print(f"✅ Saved comparative enrichment table to: {table_output_filename}")
    except Exception as e_save_csv:
        print(f"❌ Error saving comparative enrichment table to CSV: {e_save_csv}")
        traceback.print_exc()
        table_creation_error = True # Flag error if saving failed
    # --- END MODIFIED ---


else: # table_creation_error was True
    print("Skipping comparative enrichment table creation due to missing prerequisites or no run data.")
    final_enrichment_comparison_df = pd.DataFrame() # Ensure empty DF on error


# Store globally for subsequent cells
globals()['dynamic_biological_analysis_comparison_df'] = final_enrichment_comparison_df


print("\n✅ Cell 7: Comparative enrichment table creation complete.")


--- Cell 7: Compile and Format Comparative Enrichment Table (2025-04-28 21:19:11) ---
  ✅ Loaded dynamic analysis comparison table from: biological_analysis_results/Dynamic_Analysis_Across_Runs/dynamic_analysis_comparison_table.csv
Compiling data for comparative enrichment table...
✅ Comparative enrichment table DataFrame created and formatted.
✅ Saved comparative enrichment table to: biological_analysis_results/Dynamic_Biological_Analysis/dynamic_biological_enrichment_comparison_table.csv

✅ Cell 7: Comparative enrichment table creation complete.


In [9]:
# Cell 8: Generate Dynamic Biological Analysis Summary Markdown
# Description: Generates the markdown text for the biological analysis summary,
#              including the comparative enrichment table and interpretation.

import pandas as pd # Needed for to_markdown
import time
# Access global dynamic_biological_analysis_comparison_df from Cell 7

print(f"\n--- Cell 8: Generate Dynamic Biological Analysis Summary Markdown ({time.strftime('%Y-%m-%d %H:%M:%S')}) ---")

# --- Prerequisites Check ---
markdown_gen_error = False
if 'dynamic_biological_analysis_comparison_df' not in globals() or not isinstance(dynamic_biological_analysis_comparison_df, pd.DataFrame):
    print("❌ Markdown Generation Error: 'dynamic_biological_analysis_comparison_df' missing or invalid (Run Cell 7)."); markdown_gen_error = True

# --- Generate Markdown Text ---
dynamic_biological_analysis_summary_markdown = ""

if not markdown_gen_error:
    if dynamic_biological_analysis_comparison_df.empty:
        print("⚠️ Cannot generate markdown table: Comparative enrichment DataFrame is empty.")
        comparison_table_md = "*(No data available to generate table)*"
    else:
        try:
            comparison_table_md = dynamic_biological_analysis_comparison_df.to_markdown(numalign='left', stralign='left')
        except ImportError:
            comparison_table_md = "*(Table generation failed: 'tabulate' library missing. Install it.)*"
        except Exception as e_table:
            print(f"❌ Error converting DataFrame to markdown table: {e_table}")
            comparison_table_md = f"*(Table generation failed: {e_table})*"

    try:
        target_name = globals().get('TARGET_NODE_NAME', 'Target Protein')

        summary_text_lines = [f"# Biological Relevance of Dynamic Regions Across Ruleset Ablations ({target_name} Subgraph)\n"]
        summary_text_lines.append("## 1. Introduction")
        summary_text_lines.append("This analysis investigates the biological relevance of the **dynamically active regions** identified in simulations of various Network Automaton rulesets applied to the AIFM1 subgraph. By performing functional enrichment analysis on the gene sets corresponding to these dynamic regions, we assess which ruleset components contribute to generating dynamically organized patterns that are associated with known biological functions.")
        summary_text_lines.append("")
        summary_text_lines.append("The dynamically active regions were identified using a consistent metric (time-averaged absolute state change over the final 20% window) and thresholded at the 80th percentile across all nodes.")
        summary_text_lines.append("")
        summary_text_lines.append("## 2. Comparative Enrichment Analysis Table")
        summary_text_lines.append("The table below summarizes the functional enrichment results (GO Biological Process) for the dynamic region identified in each simulation run:")
        summary_text_lines.append("")
        summary_text_lines.append(comparison_table_md)
        summary_text_lines.append("")
        summary_text_lines.append("_**Table Columns:**_")
        summary_text_lines.append("- **Dynamic Region Size:** Number of nodes in the dynamic region for that run.")
        summary_text_lines.append("- **Mapped Genes:** Number of unique gene symbols mapped from the dynamic region nodes.")
        summary_text_lines.append("- **Significant Terms:** Number of GO BP terms found to be significantly enriched (Adj. P < 0.05).")
        summary_text_lines.append("- **Top Term:** The GO BP term with the most significant adjusted P-value.")
        summary_text_lines.append("- **Top Term Adj P:** The adjusted P-value for the Top Term.")
        summary_text_lines.append("")
        summary_text_lines.append("## 3. Interpretation of Biological Relevance")
        summary_text_lines.append("Based on the comparative enrichment analysis:")
        summary_text_lines.append("")

        if 'H+P (2D Ref)' in dynamic_biological_analysis_comparison_df.index:
             hp_sig_terms = dynamic_biological_analysis_comparison_df.loc['H+P (2D Ref)'].get('Significant Terms', 0)
             hp_top_term = dynamic_biological_analysis_comparison_df.loc['H+P (2D Ref)'].get('Top Term', 'N/A')
             if isinstance(hp_sig_terms, (int, float)) and hp_sig_terms > 0:
                  summary_text_lines.append(f"- **H+P Reference (2D):** The baseline H+P ruleset generated a dynamic region significantly enriched ({hp_sig_terms} terms), with '{hp_top_term}' as a top term. This confirms that the combination of Harmonic and Pheromone can produce dynamically active regions associated with relevant biology.")
             else:
                  summary_text_lines.append(f"- **H+P Reference (2D):** The dynamic region from the baseline H+P run did not show significant enrichment ({hp_sig_terms} terms found). This suggests that while it generated dynamic activity, that activity was not spatially organized in a way that strongly highlights GO BP functions under the chosen parameters and dynamic region definition.")
        else:
            summary_text_lines.append("- Interpretation Note: H+P Reference run enrichment data missing.")

        if 'H+4D-Bio (AIFM1)' in dynamic_biological_analysis_comparison_df.index:
             bio_sig_terms = dynamic_biological_analysis_comparison_df.loc['H+4D-Bio (AIFM1)'].get('Significant Terms', 0)
             bio_top_term = dynamic_biological_analysis_comparison_df.loc['H+4D-Bio (AIFM1)'].get('Top Term', 'N/A')
             if isinstance(bio_sig_terms, (int, float)) and bio_sig_terms > 0:
                  summary_text_lines.append(f"- **H+4D-Bio (AIFM1):** The dynamically active region from the 4D Biological model on AIFM1 was also significantly enriched ({bio_sig_terms} terms), with '{bio_top_term}' as a top term. This demonstrates that incorporating biologically specific state variables and rules can produce dynamically relevant regions associated with biological functions.")
             else:
                  summary_text_lines.append(f"- **H+4D-Bio (AIFM1):** The dynamic region from the 4D Biological run did not show significant enrichment ({bio_sig_terms} terms found).")
        else:
            summary_text_lines.append("- Interpretation Note: H+4D-Bio run enrichment data missing.")

        if 'P-Only (2D)' in dynamic_biological_analysis_comparison_df.index:
             p_only_sig_terms = dynamic_biological_analysis_comparison_df.loc['P-Only (2D)'].get('Significant Terms', 0)
             if isinstance(p_only_sig_terms, (int, float)):
                  summary_text_lines.append(f"- **Pheromone Only (2D):** This run, which collapsed to homogeneity, showed {p_only_sig_terms} significant terms. If this number is low or zero, it reinforces that homogeneity is not biologically relevant in this context.")
             else:
                  summary_text_lines.append("- Interpretation Note: P-Only run enrichment data missing.")

        if 'H-Only (2D)' in dynamic_biological_analysis_comparison_df.index:
             h_only_sig_terms = dynamic_biological_analysis_comparison_df.loc['H-Only (2D)'].get('Significant Terms', 0)
             if isinstance(h_only_sig_terms, (int, float)):
                  summary_text_lines.append(f"- **Harmonic Only (2D):** This run generated strong oscillations but lacked Pheromone stabilization. It showed {h_only_sig_terms} significant terms. Comparing this to the H+P run highlights if Pheromone *enhances* or *focuses* the biological signal within the dynamic region.")
             else:
                  summary_text_lines.append("- Interpretation Note: H-Only run enrichment data missing.")

        generic_ph_labels = ["H+3D-PH (Coupled)", "H+5D-PH (Coupled)", "H+5D-PH (Decoupled)"]
        generic_ph_sig_terms = {}
        for label in generic_ph_labels:
             if label in dynamic_biological_analysis_comparison_df.index:
                  sig_terms = dynamic_biological_analysis_comparison_df.loc[label].get('Significant Terms', 0)
                  if isinstance(sig_terms, (int, float)):
                       generic_ph_sig_terms[label] = sig_terms
             else:
                  generic_ph_sig_terms[label] = "Data Missing"

        if generic_ph_sig_terms:
             summary_text_lines.append("- **Generic Placeholder Runs:**")
             for label, sig_terms in generic_ph_sig_terms.items():
                  if isinstance(sig_terms, (int, float)):
                       summary_text_lines.append(f"  - The Dynamic Regions from {label} showed {sig_terms} significant terms. If this number is low compared to H+P or H+4D-Bio, it suggests generic dimensions/dynamics are less effective at highlighting biologically relevant regions.")
                  else:
                       summary_text_lines.append(f"  - Data missing for {label}.")

        summary_text_lines.append("")
        summary_text_lines.append("Conclusion: Comparing the number and nature of significant terms across runs allows us to infer which ruleset components are most effective at generating dynamically active regions that are biologically interpretable via GO enrichment.")
        summary_text_lines.append("")
        summary_text_lines.append("---")

        dynamic_biological_analysis_summary_markdown = "\n".join(summary_text_lines)

        print("✅ Dynamic biological analysis summary markdown generated.")

    except Exception as e:
        print(f"❌ Error generating dynamic biological analysis summary markdown: {e}")
        traceback.print_exc()
        markdown_gen_error = True

else:
    print("Skipping dynamic biological analysis summary markdown generation due to previous errors.")

# Store globally
globals()['dynamic_biological_analysis_summary_markdown'] = dynamic_biological_analysis_summary_markdown

print("\nCell 8: Dynamic biological analysis summary markdown generation complete.")



--- Cell 8: Generate Dynamic Biological Analysis Summary Markdown (2025-04-28 21:19:11) ---
✅ Dynamic biological analysis summary markdown generated.

Cell 8: Dynamic biological analysis summary markdown generation complete.


In [10]:
# Cell 9: Save and Display Dynamic Biological Analysis Summary Markdown
# Description: Saves the generated markdown text to a file in the dynamic biological
#              analysis results directory and prints it to the console.

import os
import time
from IPython.display import display, Markdown # For displaying markdown

print(f"\n--- Cell 9: Save and Display Dynamic Biological Analysis Summary Markdown ({time.strftime('%Y-%m-%d %H:%M:%S')}) ---")

# --- Prerequisites Check ---
save_display_error = False
if 'dynamic_biological_analysis_summary_markdown' not in globals() or not dynamic_biological_analysis_summary_markdown:
    print("❌ Cannot save/display: 'dynamic_biological_analysis_summary_markdown' missing or empty (Run Cell 8)."); save_display_error = True
if 'OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS' not in globals() or not OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS:
    print("❌ Cannot save: OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS missing (Run Cell 1)."); save_display_error = True
elif not os.path.isdir(OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS):
     print(f"❌ Cannot save: OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS directory not found: {OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS}. Check Cell 1."); save_display_error = True

# --- Execute Save and Display ---
if not save_display_error:
    summary_markdown_path = os.path.join(OUTPUT_DIR_DYNAMIC_BIO_ANALYSIS, "dynamic_biological_analysis_summary.md") # Specific filename

    try:
        with open(summary_markdown_path, 'w') as f:
            f.write(dynamic_biological_analysis_summary_markdown)
        print(f"✅ Dynamic Biological Analysis Summary saved to: {summary_markdown_path}")

        print("\n--- Displaying Dynamic Biological Analysis Summary ---")
        # Use IPython.display.Markdown to render the markdown in the notebook output
        display(Markdown(dynamic_biological_analysis_summary_markdown))
        print("--- End of Display ---")

    except Exception as e:
        print(f"❌ Error saving or displaying summary markdown: {e}")
        traceback.print_exc()
        save_display_error = True

else:
    print("Skipping save/display due to previous errors.")

print("\nCell 9: Save and Display Dynamic Biological Analysis Summary Markdown complete.")



--- Cell 9: Save and Display Dynamic Biological Analysis Summary Markdown (2025-04-28 21:19:11) ---
✅ Dynamic Biological Analysis Summary saved to: biological_analysis_results/Dynamic_Biological_Analysis/dynamic_biological_analysis_summary.md

--- Displaying Dynamic Biological Analysis Summary ---


# Biological Relevance of Dynamic Regions Across Ruleset Ablations (TargetProtein Subgraph)

## 1. Introduction
This analysis investigates the biological relevance of the **dynamically active regions** identified in simulations of various Network Automaton rulesets applied to the AIFM1 subgraph. By performing functional enrichment analysis on the gene sets corresponding to these dynamic regions, we assess which ruleset components contribute to generating dynamically organized patterns that are associated with known biological functions.

The dynamically active regions were identified using a consistent metric (time-averaged absolute state change over the final 20% window) and thresholded at the 80th percentile across all nodes.

## 2. Comparative Enrichment Analysis Table
The table below summarizes the functional enrichment results (GO Biological Process) for the dynamic region identified in each simulation run:

*(Table generation failed: 'tabulate' library missing. Install it.)*

_**Table Columns:**_
- **Dynamic Region Size:** Number of nodes in the dynamic region for that run.
- **Mapped Genes:** Number of unique gene symbols mapped from the dynamic region nodes.
- **Significant Terms:** Number of GO BP terms found to be significantly enriched (Adj. P < 0.05).
- **Top Term:** The GO BP term with the most significant adjusted P-value.
- **Top Term Adj P:** The adjusted P-value for the Top Term.

## 3. Interpretation of Biological Relevance
Based on the comparative enrichment analysis:

- **H+P Reference (2D):** The dynamic region from the baseline H+P run did not show significant enrichment (2612 terms found). This suggests that while it generated dynamic activity, that activity was not spatially organized in a way that strongly highlights GO BP functions under the chosen parameters and dynamic region definition.
- **H+4D-Bio (AIFM1):** The dynamic region from the 4D Biological run did not show significant enrichment (2700 terms found).
- Interpretation Note: P-Only run enrichment data missing.
- Interpretation Note: H-Only run enrichment data missing.

Conclusion: Comparing the number and nature of significant terms across runs allows us to infer which ruleset components are most effective at generating dynamically active regions that are biologically interpretable via GO enrichment.

---

--- End of Display ---

Cell 9: Save and Display Dynamic Biological Analysis Summary Markdown complete.
